In [ ]:
#Code 1 - Modelling for Uni modal case (Pred Z and PDI)

In [ ]:
import tensorflow as tf
import math
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import display, Math

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
np.random.seed(42)

In [ ]:
# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
print(data[" PDI"].min())
print(data[" PDI"].max())

In [ ]:
print(math.log10((((10 ** -1.632774717) + 1)*1))) # log(PDI - 1)

In [ ]:
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
print(data[" PDI"].min())
print(data[" PDI"].max())

In [ ]:
(10**(3) - 10**(0.477121255))*0.15
# print(data["Z"].min())
# print(data["Z"].max())

In [ ]:
d = data.iloc[:,97:]
print(d.head(5))
d = d.values.flatten()
d
print(d.min())
print(d.max())

In [ ]:
# for i in data.keys():
#     print(i + " : "  + str(data[i].isnull().sum()))

In [ ]:
X = data.iloc[:, 2:]  # 190 explanatory variables
y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
lis = []
for i in ['Z']:
  lis.append(y[i].unique())

lis = [item for sublist in lis for item in sublist]
for P in lis:
    c=0
    for i in y['Z']:
      if i==P:
        c = c+1
    print(P,c)

In [ ]:

# c=0
# for i in y[' PDI']:
#   if i<0.65 and i>0.4:
#     c = c+1
# print(c)

In [ ]:
plt.hist(y[" PDI"])
plt.xlabel("PDI")
plt.ylabel("Frequency")
plt.title("Frequency of PDI")
plt.show()
plt.hist(y["Z"])
plt.xlabel("Z")
plt.ylabel("Frequency")
plt.title("Frequency of Z")
plt.show()
#plt.hist(y["Z"])
#plt.show() 
# plt.hist(y_train[" PDI"])
# plt.show() 
# plt.hist(y_test[" PDI"])
# plt.show() 

In [ ]:
print(type(X_train))
print(" \n")
print(type(y_trainNP))

In [ ]:
X_train.shape

In [ ]:
data.describe(include = "all")

In [ ]:
freq = pd.read_csv('run0.dat', delim_whitespace=True,header = 0)

In [ ]:
freq = np.log10(freq['#w'])
freq

In [ ]:
Zpercentiles = np.percentile(data["Z"], [25, 50, 75])
PDIpercentiles = np.percentile(data[" PDI"], [25, 50, 75])
print(Zpercentiles)
print(PDIpercentiles)

In [ ]:
import numpy as np

def find_nearest(array, values):
    """Find the nearest value in a NumPy array."""
    array = np.asarray(array)  # Convert to NumPy array if not already
    idx = list()
    for i,value in enumerate(values):
        idx.append((np.abs(array - value)).argmin())  # Find index of the nearest value
    newarr = list()
    for val in idx:
        newarr.append(array[val])
    return array[idx]

# # Example usage
# data = np.array([1, 4, 5.5, 8, 10])
# targets = [5,11,12]
# nearest_value = find_nearest(data, targets)
# print(f"The nearest value to {target} is {nearest_value}.")

In [ ]:
Zpercentiles = np.percentile(data["Z"], [0, 25, 50, 75, 100])
PDIpercentiles = np.percentile(data[" PDI"], [0, 25, 50, 75, 100])
# print(Zpercentiles)
# print(data["Z"])
Zpercentiles = find_nearest(data["Z"], Zpercentiles)
PDIpercentiles = find_nearest(data[" PDI"], PDIpercentiles)
terms = ["Very Low","Low","Mid","High","Very High"]
for term1,i in enumerate(Zpercentiles):
    for term2,j in enumerate(PDIpercentiles):
        # needrowGprime = pd.DataFrame()
        # needrowGdprime = pd.DataFrame()
        needrowGprime = data[data["Z"]==i][data[" PDI"]==j].iloc[:, 2:97]
        needrowGdprime = data[data["Z"]==i][data[" PDI"]==j].iloc[:, 97:]
        # print(data[data["Z"]==i].iloc[:5, :7])
        # print(data[data[" PDI"]==j].iloc[:5, :7])
        #Z errors wrt PDI
        plt.plot(freq, needrowGprime.iloc[0].values,marker='o', linestyle='-', color='red',label=f"G' curve with {terms[term1]} Z and {terms[term2]} PDI")
        plt.plot(freq, needrowGdprime.iloc[0].values,marker='o', linestyle='-', color='b',label=f"G'' curve with {terms[term1]} Z and {terms[term2]} PDI")
        plt.xlabel(r'$\overline{\omega}$')
        # naming the y axis
        plt.ylabel(r"$\overline{G'} \text{ and } \overline{G''}$")
         
        # giving a title to my graph
        plt.title(f"Flow Curve at {terms[term1]} $\\overline{{Z}}$ and {terms[term2]} $\\overline{{PDI}}$")
        plt.text(min(freq), max(max(needrowGprime.iloc[0].values), max(needrowGdprime.iloc[0].values))-1, f"$\\overline{{Z}} = {round(Zpercentiles[term1],2)}$  , $\\overline{{PDI}} = {round(PDIpercentiles[term2],2)}$  ", fontsize=10, color='black')
        # Save the plot to a file
        plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\graphs\\graph_{0}{1}.png'.format(term1,term2), format='png',dpi=600)  # Save as PNG file
        plt.show()
        # print(data)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the number of rows and columns for the grid
rows, cols = 5, 5

# Create a figure with subplots arranged in a grid
fig, axs = plt.subplots(rows, cols, figsize=(12, 10))

# Load and display each image in the grid
for i in range(rows):
    for j in range(cols):
        img_path = 'C:\\Users\\Kevin\\Desktop\\Dissertation\\graphs\\graph_{0}{1}.png'.format(abs(4 - i),j)  # Path to the image file
        img = mpimg.imread(img_path)  # Load the image
        axs[i, j].imshow(img)  # Display the image
        axs[i, j].axis('off')  # Hide the axis

# Add overall x-axis and y-axis labels
fig.supxlabel(r'$\overline{PDI} \longrightarrow$', fontsize=15)
fig.supylabel(r'$\overline{Z} \longrightarrow$', fontsize=15)
fig.suptitle(f'Flow curves at different values of $\\overline{{Z}}$ and $\\overline{{PDI}}$', fontsize=15)
# # Adjust layout to prevent overlap
# plt.tight_layout()  # Adjust bottom and top margins #rect=[0, 0, 1, 0.96]
# Adjust layout to prevent overlap and increase spacing
# plt.subplots_adjust(wspace=0, hspace=0)  # Adjust space between subplots
xspace = 0.06
yspace = 0.04
space = 0.0001
plt.subplots_adjust(left=xspace, right=1-xspace, top=1-yspace, bottom=yspace, wspace=space, hspace=space)
# Adding a note using fig.text
fig.text(0.79, 0.01, f"(Note: Red curve denotes $\\overline{{G'}}$ and Blue curve denotes $\\overline{{G''}})$", ha='center', fontsize=9)
plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\graphs\\allgraph.png', format='png',dpi=300)  # Save as PNG file
plt.show()  # Display the grid of images


In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(64, input_shape=(1,190))) #return_sequences=True
    #model.add(LSTM(16))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# #metrics=['mae']
# #Define the MLP model
# def create_mlp_model(input_dim):
#     model = Sequential()
#     model.add(Dense(64, input_dim=input_dim, activation='relu'))
#     #model.add(Dropout(0.25))
#     # model.add(Dense(128, activation='relu'))
#     # model.add(Dropout(0.15))
#     model.add(Dense(16, activation='relu'))
#     model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
#     return model

# # Create the model
# model = create_mlp_model(X_train.shape[1])

# #Create the Adam optimizer with a custom learning rate
# learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
# custom_optimizer = Adam(learning_rate=learning_rate)

# # Compile the model
# model.compile(optimizer=custom_optimizer, loss='mean_squared_error')#metrics=['mae']

# # Define early stopping to prevent overfitting
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# type(X_train)
# type(y_train)
# # Train the model
# history = model.fit(X_train, y_trainNP, 
#                     validation_split=0.2, 
#                     epochs=100, 
#                     batch_size=32, 
#                     callbacks=[early_stopping])


# #Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 3.094820931437425e-05

In [ ]:
df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = df1.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model.h5")


In [ ]:
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv', index=False)

In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) = 0.0035236082972914915
#Best Z_mae (RNN) = 0.0017323339878604644

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 0.006535302287162144
#Best PDI_mae (RNN) = 0.003953151289141093

In [ ]:
#Code 2 - Error Analysis of Unimodal case (Z and PDI)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import display, Math

display(Math(r'\overline{Z} \text{ Errors vs } \overline{Z}'))


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# # Load the dataset
# data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
#data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

In [ ]:
# X = data.iloc[:, 2:]  # 190 explanatory variables
# y = data.iloc[:, :2]  # 2 numerical variables to predict

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# xcopytest = X_test

# # Standardize the data (mean=0, variance=1)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# #X_trainNP = X_train.flatten()
# y_trainNP = y_train.to_numpy()

In [ ]:
# # Load the model
# loaded_model = tf.keras.models.load_model("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model.h5")

# X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# # Make predictions with the loaded model
# y_pred_loaded = loaded_model.predict(X_test_reshaped)

# df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
# df1 = df1.reset_index(drop=False)
# df1["index"] = df1["index"] + 2
# df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
# df3 = pd.DataFrame(y_pred_loaded,columns = [x+"PRED" for x in y_test.columns])
# df = df1.join(df2).join(df3)   

In [ ]:
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv', index=False)

In [ ]:
df = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv")

In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) = 0.0035236082972914915
#Best Z_mae (RNN) = 0.0017323339878604644

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 0.006535302287162144
#Best PDI_mae (RNN) = 0.003953151289141093

In [ ]:
print(f"Highest Z error : {max(df['zerrors'])}")
print(f"Highest PDI error : {max(df['pdierrors'])}")

In [ ]:
# df["zerrorsreal"] = (10**df["Z"] - 10**df["ZPRED"]).abs()
# sorted_df = df.sort_values(by="zerrorsreal", ascending=False)
# print("Z STUFF")
# print(sorted_df["zerrorsreal"].iloc[0])
# print("Highest error Actual : ",sorted_df["Z"].iloc[0])
# print("Highest error Pred : ",sorted_df["ZPRED"].iloc[0])
# print("Error : ",sorted_df["ZPRED"].iloc[0] - sorted_df["Z"].iloc[0])
# df["pdierrorsreal"] = (10**df[" PDI"] - 10**df[" PDIPRED"]).abs()
# sorted_df = df.sort_values(by="pdierrorsreal", ascending=False)
# print("PDI STUFF")
# print(sorted_df["pdierrorsreal"].iloc[0])
# print("Highest error Actual : ",sorted_df[" PDI"].iloc[0])
# print("Highest error Pred : ",sorted_df[" PDIPRED"].iloc[0])
# print("Error : ",sorted_df[" PDIPRED"].iloc[0] - sorted_df[" PDI"].iloc[0])

In [ ]:
from IPython.display import display, Math
def code1(y1,y2,y3):
    strings = ["Z"]
    
    for s in strings:
    
        
        val = min(df[s])
        points = [val]
        while val + 0.125 < max(df[s]):
            val = val + 0.125
            points.append(val)
        points.append(max(df[s]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s]>=points[i]][df[s]<points[i+1]][s.lower()+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s]>=points[i]][df[s]<=points[i+1]][s.lower()+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        newdataZ = newdata
        pointsZ = [x + 0.0625 for x in points[:-1]]
        plt.plot(pointsZ, newdataZ,marker='o', linestyle='-', color='lime')
        # naming the x axis
        plt.xlabel(r'$\overline{Z}$')
        # naming the y axis
        plt.ylabel(r'$\overline{Z} \text{ Errors}$')
         
        # giving a title to my graph
        plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
        plt.show()
        plt.figure(figsize=(7.5, 6))
        #plt.figure(figsize=(5, 4))
        
        median_x = np.median(df["Z"])
        median_y = np.median(df["zerrors"])
        
        Q1_x, Q3_x = np.percentile(df["Z"], [25, 75])
        Q1_y, Q3_y = np.percentile(df["zerrors"], [25, 75])
        
        IQR_x = Q3_x - Q1_x
        IQR_y = Q3_y - Q1_y
        
        # Plot the density heatmap
        sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap='Spectral', fill=False, thresh=0.05, levels=2)
        
        # plotting the points 
        plt.scatter(df["Z"], df["zerrors"],s=0.1, c='black', alpha=0.5)
        
        plt.axhline(median_y, color='red', linestyle='--', linewidth=0.5, label='Median Y')
        # plt.axvline(median_x, color='red', linestyle='--', linewidth=0.5, label='Median X')
        plt.axhline(Q1_y, color='red', linestyle='--', linewidth=0.5, label='1st Quartile Y')
        plt.axhline(Q3_y, color='red', linestyle='--', linewidth=0.5, label='3rd Quartile Y')
        # plt.fill_betweenx([Q1_y, Q3_y], df["Z"].min(), df["Z"].max(), color='gray', alpha=0.2, label='IQR Y')
        # plt.fill_betweenx([Q1_x, Q3_x], df["zerrors"].min(), df["zerrors"].max(), color='lightblue', alpha=0.2, label='IQR X')
        
        # Plot the density heatmap
        #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
        plt.plot(pointsZ, newdataZ,marker='o', linestyle='-', color='lime')
        
        # Set x and y limits
        plt.xlim(df["Z"].min()-0.3, df["Z"].max()+0.3)  # Set the x-axis limit from 0 to 1
        plt.ylim(0, y1)  # Set the y-axis limit from 0 to 1
        
        # naming the x axis
        plt.xlabel(r'$\overline{Z}$')
        # naming the y axis
        plt.ylabel(r'$\overline{Z} \text{ Errors}$')
         
        # giving a title to my graph
        plt.title(r'$\overline{Z} \text{ Errors with Density Heatmap}$')
         
        # function to show the plot
        plt.show()
    
    strings = [" PDI"]
    
    for s in strings:
    
        
        val = min(df[s])
        points = [val]
        while val + 0.125 < max(df[s]):
            val = val + 0.125
            points.append(val)
        points.append(max(df[s]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s]>=points[i]][df[s]<points[i+1]]["pdi"+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s]>=points[i]][df[s]<=points[i+1]]["pdi"+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        newdataPDI = newdata
        pointsPDI = [x + 0.0625 for x in points[:-1]]
        plt.plot(pointsPDI, newdataPDI,marker='o', linestyle='-', color='lime')
        # naming the x axis
        # naming the x axis
        # naming the x axis
        plt.xlabel(r'$\overline{PDI}$')
        # naming the y axis
        plt.ylabel(r'$\overline{PDI} \text{ Errors}$')
         
        # giving a title to my graph
        plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
        plt.show()
    
        plt.figure(figsize=(7.5, 6))
        #plt.figure(figsize=(5, 4))
        median_x = np.median(df[" PDI"])
        median_y = np.median(df["pdierrors"])
        
        Q1_x, Q3_x = np.percentile(df[" PDI"], [25, 75])
        Q1_y, Q3_y = np.percentile(df["pdierrors"], [25, 75])
        
        IQR_x = Q3_x - Q1_x
        IQR_y = Q3_y - Q1_y
        # Plot the density heatmap
        sns.kdeplot(x=df[" PDI"], y=df["pdierrors"], cmap='Spectral', fill=False, thresh=0.05, levels=2)
        
        # plotting the points 
        plt.scatter(df[" PDI"], df["pdierrors"],s=0.1, c='black', alpha=0.5)
        
        plt.axhline(median_y, color='red', linestyle='--', linewidth=0.5, label='Median Y')
        # plt.axvline(median_x, color='red', linestyle='--', linewidth=0.5, label='Median X')
        plt.axhline(Q1_y, color='red', linestyle='--', linewidth=0.5, label='1st Quartile Y')
        plt.axhline(Q3_y, color='red', linestyle='--', linewidth=0.5, label='3rd Quartile Y')
        # Plot the density heatmap
        #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
        plt.plot(pointsPDI, newdataPDI,marker='o', linestyle='-', color='lime')
        plt.xlim(df[" PDI"].min()-0.3, df[" PDI"].max()+0.3)  # Set the y-axis limit from 0 to 1
        plt.ylim(0, y2)  # Set the y-axis limit from 0 to 1
        # naming the x axis
        plt.xlabel(r'$\overline{PDI}$')
        # naming the y axis
        plt.ylabel(r'$\overline{PDI} \text{ Errors}$')
         
        # giving a title to my graph
        plt.title(r'$\overline{PDI} \text{ Errors with Density Heatmap}$')
         
        # function to show the plot
        plt.show()

    strings = [["Z","pdi"], [" PDI","z"]]

    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.125 < max(df[s[0]]):
            val = val + 0.125
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        # naming the x axis
        plt.xlabel(r'$\overline{{{0}}}$'.format(s[0]))
        # naming the y axis
        plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(s[1].upper()))
         
        # giving a title to my graph
        plt.title(r'$\overline{{{1}}} \text{{ Errors vs }} \overline{{{0}}}$'.format(s[0], s[1].upper()))
        plt.show()
    
        plt.figure(figsize=(7.5, 6))
        #plt.figure(figsize=(5, 4))
        
        median_x = np.median(df[s[0]])
        median_y = np.median(df[s[1]+"errors"])
        
        Q1_x, Q3_x = np.percentile(df[s[0]], [25, 75])
        Q1_y, Q3_y = np.percentile(df[s[1]+"errors"], [25, 75])
        
        IQR_x = Q3_x - Q1_x
        IQR_y = Q3_y - Q1_y
        
        # Plot the density heatmap
        sns.kdeplot(x=df[s[0]], y=df[s[1]+"errors"], cmap='Spectral', fill=False, thresh=0.05, levels=2)
        
        # plotting the points 
        plt.scatter(df[s[0]], df[s[1]+"errors"],s=0.1, c='black', alpha=0.5)
        
        plt.axhline(median_y, color='red', linestyle='--', linewidth=0.5, label='Median Y')
        # plt.axvline(median_x, color='red', linestyle='--', linewidth=0.5, label='Median X')
        plt.axhline(Q1_y, color='red', linestyle='--', linewidth=0.5, label='1st Quartile Y')
        plt.axhline(Q3_y, color='red', linestyle='--', linewidth=0.5, label='3rd Quartile Y')
        # plt.fill_betweenx([Q1_y, Q3_y], df["Z"].min(), df["Z"].max(), color='gray', alpha=0.2, label='IQR Y')
        # plt.fill_betweenx([Q1_x, Q3_x], df["zerrors"].min(), df["zerrors"].max(), color='lightblue', alpha=0.2, label='IQR X')
        
        # Plot the density heatmap
        #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
        plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        
        # Set x and y limits
        plt.xlim(df[s[0]].min()-0.3, df[s[0]].max()+0.3)  # Set the x-axis limit from 0 to 1
        plt.ylim(0, y3)  # Set the y-axis limit from 0 to 1
        
        # naming the x axis
        plt.xlabel(r'$\overline{{{0}}}$'.format(s[0]))
        # naming the y axis
        plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(s[1].upper()))
         
        # giving a title to my graph
        plt.title(r'$\overline{{{0}}} \text{{ Errors with Density Heatmap}}$'.format(s[1].upper()))
         
        # function to show the plot
        plt.show()
    

In [ ]:
code1(0.02,0.06,0.02)

In [ ]:
#Code 3 - Changing the range of G dash values available for modelling (Unimodal case)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

In [ ]:
# X1 = data.iloc[:, 16:56]  # (G'(14) to G'(53)) (10^-3 to 10^3) (40 columns)
# X2 = data.iloc[:, 111:151]  # (G''(14) to G''(53)) (10^-3 to 10^3) (40 columns)
X1 = data.iloc[:, 22:49]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = data.iloc[:, 117:144]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)
y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
X

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(64, input_shape=(1,54))) #return_sequences=True
    #model.add(LSTM(16))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

In [ ]:
#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted.h5")


In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted.csv', index=False)

In [ ]:
#Code 4 - Error Analysis on Restricted model

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Load the dataset
# Load the dataset
#data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
#data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
df = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted.csv")  # Replace with your CSV file path

In [ ]:
# X1 = data.iloc[:, 22:49]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
# X2 = data.iloc[:, 117:144]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
# X = pd.concat([X1, X2], axis=1)
# y = data.iloc[:, :2]  # 2 numerical variables to predict

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# xcopytest = X_test

# # Standardize the data (mean=0, variance=1)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# #X_trainNP = X_train.flatten()
# y_trainNP = y_train.to_numpy()

In [ ]:
# # Load the model
# loaded_model = tf.keras.models.load_model("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted.h5")

# X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
# # Make predictions with the loaded model
# y_pred_loaded = loaded_model.predict(X_test_reshaped)

# df1 = pd.DataFrame(xcopytest,columns = X.columns)
# df1 = df1.reset_index(drop=False)
# df1["index"] = df1["index"] + 2
# df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
# df3 = pd.DataFrame(y_pred_loaded,columns = [x+"PRED" for x in y_test.columns])
# df = df1.join(df2).join(df3)   

In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) = 0.0035236082972914915
#Best Z_mae (RNN) = 0.0017323339878604644

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 0.006535302287162144
#Best PDI_mae (RNN) = 0.003953151289141093

In [ ]:
print(max(df["zerrors"]))
print(max(df["pdierrors"]))

In [ ]:
code1(0.02,0.06,0.04)

In [ ]:
#Code 5.1 - Adding random multiplicative errors to the dataset and modelling the unimodal case

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

X = data.iloc[:, 2:]  # 190 explanatory variables
y = data.iloc[:, :2]  # 2 numerical variables to predict

noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2
X = X.add(noise, axis=0) #multiplicative errors (row - wise)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(64, input_shape=(1,190))) #return_sequences=True
    #model.add(LSTM(16))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 3.094820931437425e-05

df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = df1.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
print(z_mae)

#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) (errors) = 0.007195653749104924

df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) (errors) = 0.020737031791626045

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_mulerrors.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_mulerrors.csv', index=False)

In [ ]:
#Code 5.2 - Adding random additive errors to the dataset and modelling the unimodal case

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

X = data.iloc[:, 2:]  # 190 explanatory variables
y = data.iloc[:, :2]  # 2 numerical variables to predict

noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(64, input_shape=(1,190))) #return_sequences=True
    #model.add(LSTM(16))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 3.094820931437425e-05

df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = df1.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
print(z_mae)

#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) (errors) = 0.007195653749104924

df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) (errors) = 0.020737031791626045

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_adderrors.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_adderrors.csv', index=False)

In [ ]:
#Code 5.3 - Adding random errors and random multiplicative errors to the dataset and modelling the unimodal case

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

In [ ]:
X = data.iloc[:, 2:]  # 190 explanatory variables
y = data.iloc[:, :2]  # 2 numerical variables to predict

In [ ]:
noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2

In [ ]:
noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

In [ ]:
noise

In [ ]:
X = X.add(noise, axis=0) #multiplicative errors (row - wise)

In [ ]:
noise = np.random.normal(0, 0.05, X.shape)

In [ ]:
noise

In [ ]:
X = X + noise #additive errors (element wise)

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(64, input_shape=(1,190))) #return_sequences=True
    #model.add(LSTM(16))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 3.094820931437425e-05

In [ ]:
df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = df1.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_randomerrors.h5")


In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) (errors) = 0.007195653749104924

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) (errors) = 0.020737031791626045

In [ ]:
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv', index=False)

In [ ]:
#Code 6.1 - Error analysis of unimodal with errors - Run Code 2 and change the model accordingly

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Load the dataset
df = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv")  # Replace with your CSV file path

In [ ]:
df.describe(include = "all")

In [ ]:
val = len(df[df['pdierrors']>0.06]['pdierrors'])/len(df['pdierrors'])
print(val)

In [ ]:
code1(0.02,0.06,0.04)

In [ ]:
#Code 6.2 - Error analysis of unimodal with errors (Mul, Add and Both)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
loadings = ['C:\\Users\\Kevin\\Desktop\\Dissertation\\out_mulerrors.csv',
            'C:\\Users\\Kevin\\Desktop\\Dissertation\\out_adderrors.csv',
            'C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv']
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","z"], [" PDI","pdi"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#Z errors wrt Z
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='red',label='Prediction with multiplicative and additive errors')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='b',label='Prediction with additive errors')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='purple',label='Prediction with multiplicative errors')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#PDI errors wrt PDI
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='red',label='Prediction with multiplicative and additive errors')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='b',label='Prediction with additive errors')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='purple',label='Prediction with multiplicative errors')
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv"]
loadings = ['C:\\Users\\Kevin\\Desktop\\Dissertation\\out_mulerrors.csv',
            'C:\\Users\\Kevin\\Desktop\\Dissertation\\out_adderrors.csv',
            'C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv']
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","pdi"], [" PDI","z"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#PDI errors wrt Z
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='red',label='Prediction with multiplicative and additive errors')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='b',label='Prediction with additive errors')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='purple',label='Prediction with multiplicative errors')
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#Z errors wrt PDI
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='red',label='Prediction with multiplicative and additive errors')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='b',label='Prediction with additive errors')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='purple',label='Prediction with multiplicative errors')
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#All the error graphs together (Original, Restricted, Error induced)

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","z"], [" PDI","pdi"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#Z
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#PDI
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","pdi"], [" PDI","z"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#PDI errors wrt Z
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#Z errors wrt PDI
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$')
 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#Code 7 - Analyzing errors in modelling Z and PDI with restricted ranges (low, mid, high)

In [ ]:
import tensorflow as tf
import math
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

In [ ]:
X = data.iloc[:, 2:]  # 190 explanatory variables

In [ ]:
cross1 = []
flag = 0 
for i, row in X.iterrows():
    for j in range(len(row)-95):
        #print(row)
        if row.iloc[j] > row.iloc[j+95] and flag==0:
            flag = 1
            cross1.append(j)
            break
    if flag == 0:
        cross1.append(-1)
    flag = 0

In [ ]:
print(cross1)

In [ ]:
cross2 = []
flag = 0 
for i, row in X.iterrows():
    for j in range(len(row)-95-1,-1,-1):
        #print(row)
        if row.iloc[j] > row.iloc[j+95] and flag==0:
            flag = 1
            cross2.append(j)
            break
    if flag == 0:
        cross2.append(-1)
    flag = 0

In [ ]:
print(cross2)

In [ ]:
frequency_dict = {}

for item in cross1:
    if item in frequency_dict:
        frequency_dict[item] += 1
    else:
        frequency_dict[item] = 1

print(frequency_dict)

In [ ]:
frequency_dict = {}

for item in cross2:
    if item in frequency_dict:
        frequency_dict[item] += 1
    else:
        frequency_dict[item] = 1

print(frequency_dict)

In [ ]:
cross1copy = cross1.copy()
cross2copy = cross2.copy()
cross1copy = [item for item in cross1copy if item != -1]
cross2copy = [item for item in cross2copy if item != -1]
print(sum(cross1copy)/len(cross1copy))
print(sum(cross2copy)/len(cross2copy))

In [ ]:
#So 1st split is : 0 to 46
# 2nd split is : 47 to 83
# 3rd split is 84 to 94

In [ ]:
#Lower range

In [ ]:
#X = data.iloc[:, 2:]  # 190 explanatory variables
X1 = X.iloc[:, 0:46+1]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = X.iloc[:, 95:95+47]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)
y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
X

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,94),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

In [ ]:
#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted_low.h5")

In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv', index=False)

In [ ]:
#Middle Range

In [ ]:
X = data.iloc[:, 2:]  # 190 explanatory variables
X1 = X.iloc[:, 47:83+1]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = X.iloc[:, 95 + 47:95+83+1]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)
y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,74),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted_mid.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv', index=False)

In [ ]:
#High Range

In [ ]:
X = data.iloc[:, 2:]  # 190 explanatory variables
X1 = X.iloc[:, 84:94+1]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = X.iloc[:, 95 + 84:95+94+1]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)
y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,22),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
z_mae

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

In [ ]:
df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
pdi_mae

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted_high.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv', index=False)

In [ ]:
#Code 8 - Analyzing the errors of restricted modelling Low, Med and High

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","z"], [" PDI","pdi"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#Z errors wrt Z
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range')
plt.plot(POINTS[7], NEWDATA[7],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#PDI errors wrt PDI
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range')
plt.plot(POINTS[8], NEWDATA[8],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range')
# naming the x axis
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","pdi"], [" PDI","z"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#PDI errors wrt Z
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range')
plt.plot(POINTS[7], NEWDATA[7],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#Z errors wrt PDI
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range')
plt.plot(POINTS[8], NEWDATA[8],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range')
# naming the x axis
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#Code 9 - Adding errors to the restricted frequency bands and seeing how model reacts

In [ ]:
#Low

In [ ]:
import tensorflow as tf
import math
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
X = data.iloc[:, 2:]  # 190 explanatory variables


#X = data.iloc[:, 2:]  # 190 explanatory variables
X1 = X.iloc[:, 0:46+1]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = X.iloc[:, 95:95+47]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)

noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,94),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
print(z_mae)

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
print(pdi_mae)

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted_low_errors.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low_errors.csv', index=False)

In [ ]:
#Mid

In [ ]:
import tensorflow as tf
import math
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
X = data.iloc[:, 2:]  # 190 explanatory variables

X1 = X.iloc[:, 47:83+1]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = X.iloc[:, 95 + 47:95+83+1]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)

noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,74),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
print(z_mae)

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
print(pdi_mae)

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted_mid_errors.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid_errors.csv', index=False)

In [ ]:
#High

In [ ]:
import tensorflow as tf
import math
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
X = data.iloc[:, 2:]  # 190 explanatory variables

X1 = X.iloc[:, 84:94+1]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = X.iloc[:, 95 + 84:95+94+1]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)

noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

y = data.iloc[:, :2]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,22),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.00015784671995788813

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
print(z_mae)

#Best Z_mae (MLP) =
#Best Z_mae (RNN) = 0.0017323339878604644
#Best Z_mae (RNN) = 0.003330880872368445 (restricted)

df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
print(pdi_mae)

#Best PDI_mae (MLP) = 
#Best PDI_mae (RNN) = 0.003953151289141093
#Best PDI_mae (RNN) = 0.005428341752105807 (restricted)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_restricted_high_errors.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high_errors.csv', index=False)

In [ ]:
# Code 10 - Analyzing the difference in errors in prediction of limited freq trained data (low, mid, high)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv",
            "C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv",
           "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low_errors.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid_errors.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high_errors.csv",
           "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","z"], [" PDI","pdi"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#Z errors wrt Z
plt.plot(POINTS[13], NEWDATA[13],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range with errors')
plt.plot(POINTS[11], NEWDATA[11],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range with errors')
plt.plot(POINTS[9], NEWDATA[9],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range with errors')
plt.plot(POINTS[15], NEWDATA[15],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range with errors')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#PDI errors wrt PDI
plt.plot(POINTS[14], NEWDATA[14],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range with errors')
plt.plot(POINTS[12], NEWDATA[12],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range with errors')
plt.plot(POINTS[10], NEWDATA[10],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range with errors')
plt.plot(POINTS[16], NEWDATA[16],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range with errors')
# naming the x axis
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv"]
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high.csv",
            "C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv",
           "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_low_errors.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_mid_errors.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted_high_errors.csv",
           "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","pdi"], [" PDI","z"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#PDI errors wrt Z
plt.plot(POINTS[13], NEWDATA[13],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range with errors')
plt.plot(POINTS[11], NEWDATA[11],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range with errors')
plt.plot(POINTS[9], NEWDATA[9],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range with errors')
plt.plot(POINTS[15], NEWDATA[15],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range with errors')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#Z errors wrt PDI
plt.plot(POINTS[14], NEWDATA[14],marker='o', linestyle='-', color='red',label='Prediction with restricted high frequency range with errors')
plt.plot(POINTS[12], NEWDATA[12],marker='o', linestyle='-', color='b',label='Prediction with restricted mid frequency range with errors')
plt.plot(POINTS[10], NEWDATA[10],marker='o', linestyle='-', color='purple',label='Prediction with restricted low frequency range with errors')
plt.plot(POINTS[16], NEWDATA[16],marker='o', linestyle='-', color='lime',label='Prediction with full frequency range with errors')
# naming the x axis
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#Code 11 - Unclean data predictions on cleanly trained model and vice versa 

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

def trying(moodel, deteset, texts):
    # Load the dataset
    data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
    data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
    
    X = data.iloc[:, 2:]  # 190 explanatory variables
    y = data.iloc[:, :2]  # 2 numerical variables to predict

    if deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv":
        noise = np.random.normal(0, 0.1, (202500, 1))
        # noise2 = np.random.normal(1, 0, (202500, 1))
        # noise3 = noise + noise2
        X = X.add(noise, axis=0) #multiplicative errors (row - wise)
        noise = np.random.normal(0, 0.05, X.shape)
        X = X + noise #additive errors (element wise)
        
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    xcopytest = X_test
    
    # Standardize the data (mean=0, variance=1)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    #X_trainNP = X_train.flatten()
    y_trainNP = y_train.to_numpy()
    
    # Load the model
    loaded_model = tf.keras.models.load_model(moodel)
    
    X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    # Make predictions with the loaded model
    y_pred_loaded = loaded_model.predict(X_test_reshaped)
    
    df1 = pd.DataFrame(xcopytest,columns = X.columns)
    df1 = df1.reset_index(drop=False)
    df1["index"] = df1["index"] + 2
    df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
    df3 = pd.DataFrame(y_pred_loaded,columns = [x+"PRED" for x in y_test.columns])
    df = df1.join(df2).join(df3)   
    
    df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
    z_mae =sum(df["zerrors"])/len(df["zerrors"])
    print(z_mae)
    
    #Best Z_mae (MLP) = 0.0035236082972914915
    #Best Z_mae (RNN) = 0.0017323339878604644
    
    df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
    pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
    print(pdi_mae)
    
    #Best PDI_mae (MLP) = 
    #Best PDI_mae (RNN) = 0.003953151289141093
    #Best PDI_mae (RNN) = 0.005428341752105807 (restricted)
    if deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv":
        deteset2 = "C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv"
    elif deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv":
        deteset2 = "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv"

    loadings = [0,deteset2]
    count = 0
    NEWDATA = [[]]
    POINTS = [[]]
    strings = [["Z","z"], [" PDI","pdi"],["Z","pdi"], [" PDI","z"]]
    for l in loadings:
        if l!=0:
            df = pd.read_csv(l)
        for s in strings:
        
            
            val = min(df[s[0]])
            points = [val]
            while val + 0.0625*2 < max(df[s[0]]):
                val = val + 0.0625*2
                points.append(val)
            points.append(max(df[s[0]]))
        
            #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
            newdata = []
            for i,c in enumerate(points[:-1]):
                if i!=len(points)-1:
                    arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                    newdata.append(sum(arr)/len(arr))
                else:
                    arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                    newdata.append(sum(arr)/len(arr))
            newdata
            #newdataZ = newdata
            points = [x + 0.0625 for x in points[:-1]]
            POINTS.append(points)
            NEWDATA.append(newdata)
            #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
            #plt.show()

    if deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv":
        extra = 0
    else:
        extra = 2

    #Z errors wrt Z
    plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='red',label=texts[0+extra])
    plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='lime',label=texts[1+extra])
    # naming the x axis
    plt.xlabel(r'$\overline{Z}$')
    # naming the y axis
    plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
    # giving a title to my graph
    plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
    plt.legend()
    plt.show()

    #PDI errors wrt PDI
    plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='red',label=texts[0+extra])
    plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='lime',label=texts[1+extra])
    # naming the x axis
    plt.xlabel(r'$\overline{PDI}$')
    # naming the y axis
    plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
    # giving a title to my graph
    plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
    plt.legend()
    plt.show()
    
    #PDI errors wrt Z
    plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='red',label=texts[0+extra])
    plt.plot(POINTS[7], NEWDATA[7],marker='o', linestyle='-', color='lime',label=texts[1+extra])
    # naming the x axis
    plt.xlabel(r'$\overline{Z}$')
    # naming the y axis
    plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
    # giving a title to my graph
    plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{Z}$')
    plt.legend()
    plt.show()
    
    #Z errors wrt PDI
    plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='red',label=texts[0+extra])
    plt.plot(POINTS[8], NEWDATA[8],marker='o', linestyle='-', color='lime',label=texts[1+extra])
    # naming the x axis
    plt.xlabel(r'$\overline{PDI}$')
    # naming the y axis
    plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
    # giving a title to my graph
    plt.title(r'$\overline{Z} \text{ Errors vs } \overline{PDI}$')
    plt.legend()
    plt.show()

In [ ]:
textss = ["Prediction of clean model on unclean data","Prediction of clean model on clean data","Prediction of unclean model on clean data","Prediction of unclean model on unclean data"]
#textss = ["Prediction of unclean data on clean model","Prediction of clean data on clean model","Prediction of clean data on unclean model","Prediction of unclean data on unclean model"]

In [ ]:
trying("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model.h5", "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv",textss)

In [ ]:
trying("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_randomerrors.h5", "C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv",textss)

In [ ]:
#Code 12 - Working on "Real life data", ie restricted freq range and noise

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\uni_PE_wint_2_low_uniform_200000_training.csv")  # Replace with your CSV file path
data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))

#X = data.iloc[:, 2:]  # 190 explanatory variables
y = data.iloc[:, :2]  # 2 numerical variables to predict

X1 = data.iloc[:, 22:49]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = data.iloc[:, 117:144]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)


noise = np.random.normal(0, 0.1, (202500, 1))
# noise2 = np.random.normal(1, 0, (202500, 1))
# noise3 = noise + noise2
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)
        
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()


In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(100, input_shape=(1,54),return_sequences=True)) #return_sequences=True
    model.add(LSTM(32))
    #model.add(Dropout(0.25))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.15))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Load the model
#loaded_model = tf.keras.models.load_model(model)

# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

df1 = pd.DataFrame(xcopytest,columns = X.columns)
df1 = df1.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred_loaded,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["zerrors"] = (df["Z"] - df["ZPRED"]).abs()
z_mae =sum(df["zerrors"])/len(df["zerrors"])
print(z_mae)

#Best Z_mae (MLP) = 0.0035236082972914915
#Best Z_mae (RNN) = 0.0017323339878604644

df["pdierrors"] = (df[" PDI"] - df[" PDIPRED"]).abs()
pdi_mae =sum(df["pdierrors"])/len(df["pdierrors"])
print(pdi_mae)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_realdata.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_realdata.csv', index=False)

In [ ]:
#Code 13 - Error analysis on pseudo real life data

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

df = pd.read_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_realdata.csv')



In [ ]:
df.describe(include = "all")

In [ ]:
print(len(df[df['zerrors']>0.06]['zerrors'])/len(df['zerrors']))
print(len(df[df['pdierrors']>0.06]['pdierrors'])/len(df['pdierrors']))

In [ ]:
code1(0.08,0.12,0.12)

In [ ]:
#All the error graphs together

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_restricted.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_randomerrors.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_realdata.csv"]
count = 0
NEWDATA = [[]]
POINTS = [[]]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    strings = [["Z","z"], [" PDI","pdi"],["Z","pdi"], [" PDI","z"]]
    
    for s in strings:
    
        
        val = min(df[s[0]])
        points = [val]
        while val + 0.0625*2 < max(df[s[0]]):
            val = val + 0.0625*2
            points.append(val)
        points.append(max(df[s[0]]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s[0]]>=points[i]][df[s[0]]<=points[i+1]][s[1]+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        #newdataZ = newdata
        points = [x + 0.0625 for x in points[:-1]]
        POINTS.append(points)
        NEWDATA.append(newdata)
        #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
        #plt.show()

In [ ]:
#Z
plt.plot(POINTS[13], NEWDATA[13],marker='o', linestyle='-', color='purple',label='Prediction with restricted frequency and artificial errors induced')
plt.plot(POINTS[9], NEWDATA[9],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[5], NEWDATA[5],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[1], NEWDATA[1],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#PDI
plt.plot(POINTS[14], NEWDATA[14],marker='o', linestyle='-', color='purple',label='Prediction with restricted frequency and artificial errors induced')
plt.plot(POINTS[10], NEWDATA[10],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[6], NEWDATA[6],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[2], NEWDATA[2],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
# naming the x axis
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#PDI errors wrt Z
plt.plot(POINTS[15], NEWDATA[15],marker='o', linestyle='-', color='purple',label='Prediction with restricted frequency and artificial errors induced')
plt.plot(POINTS[11], NEWDATA[11],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[7], NEWDATA[7],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[3], NEWDATA[3],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
# naming the x axis
plt.xlabel(r'$\overline{Z}$')
# naming the y axis
plt.ylabel(r'$\overline{PDI} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{PDI} \text{ Errors vs } \overline{Z}$')
plt.legend()
plt.show()

In [ ]:
#Z errors wrt PDI
plt.plot(POINTS[16], NEWDATA[16],marker='o', linestyle='-', color='purple',label='Prediction with restricted frequency and artificial errors induced')
plt.plot(POINTS[12], NEWDATA[12],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
plt.plot(POINTS[8], NEWDATA[8],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
plt.plot(POINTS[4], NEWDATA[4],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
# naming the x axis
plt.xlabel(r'$\overline{PDI}$')
# naming the y axis
plt.ylabel(r'$\overline{Z} \text{ Errors}$') 
# giving a title to my graph
plt.title(r'$\overline{Z} \text{ Errors vs } \overline{PDI}$')
plt.legend()
plt.show()

In [ ]:
#Code 14 - Modelling for Bi - modal case (Pred Z1, Z2, PDI1, PDI2 and Phi)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
np.random.seed(42)

In [ ]:
# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\bi_PE_5param_wint_2_400000_training.csv")  # Replace with your CSV file path

In [ ]:
data

In [ ]:
X = data.iloc[:, 5:]  # 190 explanatory variables
y = data.iloc[:, :5]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
X_train.shape

In [ ]:
# lis = []
# for i in ['Zs']:
#   lis.append(y[i].unique())

# lis = [item for sublist in lis for item in sublist]
# for P in lis:
#     c=0
#     for i in y['Zs']:
#       if i==P:
#         c = c+1
#     print(P,c)

In [ ]:
# y["sl"] = y["Zs"] + y["Zl"]
# plt.hist(y["sl"])
# plt.show() 

In [ ]:
data.describe(include = "all")

In [ ]:
d = data.iloc[:,:5]
print(d)

In [ ]:
freq = pd.read_csv('run0.dat', delim_whitespace=True,header = 0)
freq = np.log10(freq['#w'])
freq

In [ ]:
#Zs vs PDI_s
import numpy as np

import numpy as np
def nearest(points,reference_points):
    # Define the set of 5D points (each row is a point)
    # points = np.array([
    #     [1, 2, 3, 4, 5],
    #     [2, 3, 4, 5, 6],
    #     [0, 0, 0, 0, 0],
    #     [1, 1, 1, 1, 1],
    #     [5, 5, 5, 5, 5]
    # ])
    points = np.asarray(points)
    # Define a reference point (e.g., the mean of all points or another specific point)
    # Here, using the mean point as the reference
    # reference_point = np.mean(points, axis=0)
    for i,reference_point in enumerate(reference_points):
        
        # Calculate the absolute error for each point
        absolute_errors = np.sum(np.abs(points - reference_point), axis=1)
        #print(absolute_errors)
        # Find the index of the point with the least absolute error
        min_error_index = np.argmin(absolute_errors)
        #print(min_error_index)
        # Get the 5D point with the least absolute error
        #print(points)
        point_with_least_error = points[min_error_index,:]
        # print(i)
        # print(point_with_least_error)
        Zpercentiles[i] = point_with_least_error[0]
        otherZpercentiles[i] = point_with_least_error[1]
        PDIpercentiles[i] = point_with_least_error[2]
        otherPDIpercentiles[i] = point_with_least_error[3]
        phipercentiles[i] = point_with_least_error[4]
    
    # print("Reference Point:", reference_point)
    # print("Point with the least absolute error:", point_with_least_error)
    # print("Minimum absolute error:", absolute_errors[min_error_index])

# reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
# nearest(data[,:5],reference_points)

# # Example usage
# data = np.array([1, 4, 5.5, 8, 10])
# targets = [5,11,12]
# nearest_value = find_nearest(data, targets)
# print(f"The nearest value to {target} is {nearest_value}.")
y = "Zs"
x = "PDI_s"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
Zpercentiles = np.percentile(data[y], [0, 25, 50, 75, 100]) #ypercentiles
Zpercentiles = [element for element in Zpercentiles for _ in range(5)]
# Zpercentiles = Zpercentilesr
PDIpercentiles = np.percentile(data[x], [0, 25, 50, 75, 100]) #xpercentiles
PDIpercentiles = np.tile(PDIpercentiles, 5)
# print(PDIpercentilesr)
# PDIpercentiles = PDIpercentilesr
#other percentiles 
phipercentiles = np.percentile(data["phiL"], [0, 25, 50, 75, 100]) #xpercentiles
otherZpercentiles = np.percentile(data["Zl"], [0, 25, 50, 75, 100]) #ypercentiles
otherPDIpercentiles = np.percentile(data["PDI_l"], [0, 25, 50, 75, 100]) #xpercentiles
# print(otherZpercentiles[2])
otherZpercentiles = [otherZpercentiles[2]] * 25
# print(otherZpercentiles)
otherPDIpercentiles = [otherPDIpercentiles[2]] * 25
phipercentiles = [phipercentiles[2]] * 25
# print(Zpercentiles)
# print(data["Z"])
# Zpercentiles = find_nearest(data[y], Zpercentiles)
# PDIpercentiles = find_nearest(data[x], PDIpercentiles)
# otherZpercentiles = find_nearest(data["Zl"], otherZpercentiles)
# otherPDIpercentiles = find_nearest(data["PDI_l"], otherPDIpercentiles)
# phipercentiles = find_nearest(data["phiL"], phipercentiles)
# print(len(Zpercentiles), len(otherZpercentiles),len( PDIpercentiles), len(otherPDIpercentiles), len(phipercentiles))
reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
d = data.iloc[:,:5]
nearest(d,reference_points)
# print(Zpercentiles[5], otherZpercentiles[5], PDIpercentiles[5], otherPDIpercentiles[5], phipercentiles[5])
# print(Zpercentiles)
terms = ["Very Low","Low","Mid","High","Very High"]
for term1,i in enumerate(Zpercentiles):
    # needrowGprime = pd.DataFrame()
    # needrowGdprime = pd.DataFrame()
    needrowGprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 5:100]
    needrowGdprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 100:]
    # print(needrowGprime)
    # print(needrowGdprime)
    # print(data[data["Z"]==i].iloc[:5, :7])
    # print(data[data[" PDI"]==j].iloc[:5, :7])
    #Z errors wrt PDI
    print(len(freq),len(needrowGprime.iloc[0].values),len(needrowGdprime.iloc[0].values))
    plt.plot(freq, needrowGprime.iloc[0].values,marker='o', linestyle='-', color='red',label=f"G' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.plot(freq, needrowGdprime.iloc[0].values,marker='o', linestyle='-', color='b',label=f"G'' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.xlabel(r'$\overline{\omega}$')
    # naming the y axis
    plt.ylabel(r"$\overline{G'} \text{ and } \overline{G''}$")
     
    # giving a title to my graph
    plt.title(f"Flow Curve at {terms[int(term1/5)]} $\\overline{{{dict[y]}}}$ and {terms[term1%5]} $\\overline{{{dict[x]}}}$")
    plt.text(min(freq), max(max(needrowGprime.iloc[0].values), max(needrowGdprime.iloc[0].values))-1, f"$\\overline{{{dict[y]}}} = {round(Zpercentiles[term1],2)}$  , $\\overline{{{dict[x]}}} = {round(PDIpercentiles[term1],2)}$  ", fontsize=10, color='black')
    # Save the plot to a file
    plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(int(term1/5),term1%5,y,x), format='png',dpi = 300)  # Save as PNG file #dpi = 600
    plt.show()
    # print(data)


In [ ]:
#Zs vs PDI_l
import numpy as np

import numpy as np
def nearest(points,reference_points):
    # Define the set of 5D points (each row is a point)
    # points = np.array([
    #     [1, 2, 3, 4, 5],
    #     [2, 3, 4, 5, 6],
    #     [0, 0, 0, 0, 0],
    #     [1, 1, 1, 1, 1],
    #     [5, 5, 5, 5, 5]
    # ])
    points = np.asarray(points)
    # Define a reference point (e.g., the mean of all points or another specific point)
    # Here, using the mean point as the reference
    # reference_point = np.mean(points, axis=0)
    for i,reference_point in enumerate(reference_points):
        
        # Calculate the absolute error for each point
        absolute_errors = np.sum(np.abs(points - reference_point), axis=1)
        #print(absolute_errors)
        # Find the index of the point with the least absolute error
        min_error_index = np.argmin(absolute_errors)
        #print(min_error_index)
        # Get the 5D point with the least absolute error
        #print(points)
        point_with_least_error = points[min_error_index,:]
        # print(i)
        # print(point_with_least_error)
        Zpercentiles[i] = point_with_least_error[0]
        otherZpercentiles[i] = point_with_least_error[1]
        PDIpercentiles[i] = point_with_least_error[2]
        otherPDIpercentiles[i] = point_with_least_error[3]
        phipercentiles[i] = point_with_least_error[4]
    
    # print("Reference Point:", reference_point)
    # print("Point with the least absolute error:", point_with_least_error)
    # print("Minimum absolute error:", absolute_errors[min_error_index])

# reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
# nearest(data[,:5],reference_points)

# # Example usage
# data = np.array([1, 4, 5.5, 8, 10])
# targets = [5,11,12]
# nearest_value = find_nearest(data, targets)
# print(f"The nearest value to {target} is {nearest_value}.")
y = "Zs"
x = "PDI_l"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
Zpercentiles = np.percentile(data[y], [0, 25, 50, 75, 100]) #ypercentiles
Zpercentiles = [element for element in Zpercentiles for _ in range(5)]
# Zpercentiles = Zpercentilesr
otherPDIpercentiles = np.percentile(data[x], [0, 25, 50, 75, 100]) #xpercentiles
otherPDIpercentiles = np.tile(otherPDIpercentiles, 5)
# print(PDIpercentilesr)
# PDIpercentiles = PDIpercentilesr
#other percentiles 
phipercentiles = np.percentile(data["phiL"], [0, 25, 50, 75, 100]) #xpercentiles
otherZpercentiles = np.percentile(data["Zl"], [0, 25, 50, 75, 100]) #ypercentiles
PDIpercentiles = np.percentile(data["PDI_s"], [0, 25, 50, 75, 100]) #xpercentiles
# print(otherZpercentiles[2])
otherZpercentiles = [otherZpercentiles[2]] * 25
# print(otherZpercentiles)
PDIpercentiles = [PDIpercentiles[2]] * 25
phipercentiles = [phipercentiles[2]] * 25
# print(Zpercentiles)
# print(data["Z"])
# Zpercentiles = find_nearest(data[y], Zpercentiles)
# PDIpercentiles = find_nearest(data[x], PDIpercentiles)
# otherZpercentiles = find_nearest(data["Zl"], otherZpercentiles)
# otherPDIpercentiles = find_nearest(data["PDI_l"], otherPDIpercentiles)
# phipercentiles = find_nearest(data["phiL"], phipercentiles)
# print(len(Zpercentiles), len(otherZpercentiles),len( PDIpercentiles), len(otherPDIpercentiles), len(phipercentiles))
reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
d = data.iloc[:,:5]
nearest(d,reference_points)
# print(Zpercentiles[5], otherZpercentiles[5], PDIpercentiles[5], otherPDIpercentiles[5], phipercentiles[5])
# print(Zpercentiles)
terms = ["Very Low","Low","Mid","High","Very High"]
for term1,i in enumerate(Zpercentiles):
    # needrowGprime = pd.DataFrame()
    # needrowGdprime = pd.DataFrame()
    needrowGprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 5:100]
    needrowGdprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 100:]
    # print(needrowGprime)
    # print(needrowGdprime)
    # print(data[data["Z"]==i].iloc[:5, :7])
    # print(data[data[" PDI"]==j].iloc[:5, :7])
    #Z errors wrt PDI
    print(len(freq),len(needrowGprime.iloc[0].values),len(needrowGdprime.iloc[0].values))
    plt.plot(freq, needrowGprime.iloc[0].values,marker='o', linestyle='-', color='red',label=f"G' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.plot(freq, needrowGdprime.iloc[0].values,marker='o', linestyle='-', color='b',label=f"G'' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.xlabel(r'$\overline{\omega}$')
    # naming the y axis
    plt.ylabel(r"$\overline{G'} \text{ and } \overline{G''}$")
     
    # giving a title to my graph
    plt.title(f"Flow Curve at {terms[int(term1/5)]} $\\overline{{{dict[y]}}}$ and {terms[term1%5]} $\\overline{{{dict[x]}}}$")
    plt.text(min(freq), max(max(needrowGprime.iloc[0].values), max(needrowGdprime.iloc[0].values))-1, f"$\\overline{{{dict[y]}}} = {round(Zpercentiles[term1],2)}$  , $\\overline{{{dict[x]}}} = {round(PDIpercentiles[term1],2)}$  ", fontsize=10, color='black')
    # Save the plot to a file
    plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(int(term1/5),term1%5,y,x), format='png',dpi = 300)  # Save as PNG file #dpi = 600
    plt.show()
    # print(data)


In [ ]:
#Zl vs PDI_l
import numpy as np

import numpy as np
def nearest(points,reference_points):
    # Define the set of 5D points (each row is a point)
    # points = np.array([
    #     [1, 2, 3, 4, 5],
    #     [2, 3, 4, 5, 6],
    #     [0, 0, 0, 0, 0],
    #     [1, 1, 1, 1, 1],
    #     [5, 5, 5, 5, 5]
    # ])
    points = np.asarray(points)
    # Define a reference point (e.g., the mean of all points or another specific point)
    # Here, using the mean point as the reference
    # reference_point = np.mean(points, axis=0)
    for i,reference_point in enumerate(reference_points):
        
        # Calculate the absolute error for each point
        absolute_errors = np.sum(np.abs(points - reference_point), axis=1)
        #print(absolute_errors)
        # Find the index of the point with the least absolute error
        min_error_index = np.argmin(absolute_errors)
        #print(min_error_index)
        # Get the 5D point with the least absolute error
        #print(points)
        point_with_least_error = points[min_error_index,:]
        # print(i)
        # print(point_with_least_error)
        Zpercentiles[i] = point_with_least_error[0]
        otherZpercentiles[i] = point_with_least_error[1]
        PDIpercentiles[i] = point_with_least_error[2]
        otherPDIpercentiles[i] = point_with_least_error[3]
        phipercentiles[i] = point_with_least_error[4]
    
    # print("Reference Point:", reference_point)
    # print("Point with the least absolute error:", point_with_least_error)
    # print("Minimum absolute error:", absolute_errors[min_error_index])

# reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
# nearest(data[,:5],reference_points)

# # Example usage
# data = np.array([1, 4, 5.5, 8, 10])
# targets = [5,11,12]
# nearest_value = find_nearest(data, targets)
# print(f"The nearest value to {target} is {nearest_value}.")
y = "Zl"
x = "PDI_l"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
otherZpercentiles = np.percentile(data[y], [0, 25, 50, 75, 100]) #ypercentiles
otherZpercentiles = [element for element in otherZpercentiles for _ in range(5)]
# Zpercentiles = Zpercentilesr
otherPDIpercentiles = np.percentile(data[x], [0, 25, 50, 75, 100]) #xpercentiles
otherPDIpercentiles = np.tile(otherPDIpercentiles, 5)
# print(PDIpercentilesr)
# PDIpercentiles = PDIpercentilesr
#other percentiles 
phipercentiles = np.percentile(data["phiL"], [0, 25, 50, 75, 100]) #xpercentiles
Zpercentiles = np.percentile(data["Zs"], [0, 25, 50, 75, 100]) #ypercentiles
PDIpercentiles = np.percentile(data["PDI_s"], [0, 25, 50, 75, 100]) #xpercentiles
# print(otherZpercentiles[2])
Zpercentiles = [Zpercentiles[2]] * 25
# print(otherZpercentiles)
PDIpercentiles = [PDIpercentiles[2]] * 25
phipercentiles = [phipercentiles[2]] * 25
# print(Zpercentiles)
# print(data["Z"])
# Zpercentiles = find_nearest(data[y], Zpercentiles)
# PDIpercentiles = find_nearest(data[x], PDIpercentiles)
# otherZpercentiles = find_nearest(data["Zl"], otherZpercentiles)
# otherPDIpercentiles = find_nearest(data["PDI_l"], otherPDIpercentiles)
# phipercentiles = find_nearest(data["phiL"], phipercentiles)
# print(len(Zpercentiles), len(otherZpercentiles),len( PDIpercentiles), len(otherPDIpercentiles), len(phipercentiles))
reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
d = data.iloc[:,:5]
nearest(d,reference_points)
# print(Zpercentiles[5], otherZpercentiles[5], PDIpercentiles[5], otherPDIpercentiles[5], phipercentiles[5])
# print(Zpercentiles)
terms = ["Very Low","Low","Mid","High","Very High"]
for term1,i in enumerate(Zpercentiles):
    # needrowGprime = pd.DataFrame()
    # needrowGdprime = pd.DataFrame()
    needrowGprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 5:100]
    needrowGdprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 100:]
    # print(needrowGprime)
    # print(needrowGdprime)
    # print(data[data["Z"]==i].iloc[:5, :7])
    # print(data[data[" PDI"]==j].iloc[:5, :7])
    #Z errors wrt PDI
    print(len(freq),len(needrowGprime.iloc[0].values),len(needrowGdprime.iloc[0].values))
    plt.plot(freq, needrowGprime.iloc[0].values,marker='o', linestyle='-', color='red',label=f"G' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.plot(freq, needrowGdprime.iloc[0].values,marker='o', linestyle='-', color='b',label=f"G'' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.xlabel(r'$\overline{\omega}$')
    # naming the y axis
    plt.ylabel(r"$\overline{G'} \text{ and } \overline{G''}$")
     
    # giving a title to my graph
    plt.title(f"Flow Curve at {terms[int(term1/5)]} $\\overline{{{dict[y]}}}$ and {terms[term1%5]} $\\overline{{{dict[x]}}}$")
    plt.text(min(freq), max(max(needrowGprime.iloc[0].values), max(needrowGdprime.iloc[0].values))-1, f"$\\overline{{{dict[y]}}} = {round(Zpercentiles[term1],2)}$  , $\\overline{{{dict[x]}}} = {round(PDIpercentiles[term1],2)}$  ", fontsize=10, color='black')
    # Save the plot to a file
    plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(int(term1/5),term1%5,y,x), format='png',dpi = 300)  # Save as PNG file #dpi = 600
    plt.show()
    # print(data)


In [ ]:
#Zl vs PDI_s
import numpy as np

import numpy as np
def nearest(points,reference_points):
    # Define the set of 5D points (each row is a point)
    # points = np.array([
    #     [1, 2, 3, 4, 5],
    #     [2, 3, 4, 5, 6],
    #     [0, 0, 0, 0, 0],
    #     [1, 1, 1, 1, 1],
    #     [5, 5, 5, 5, 5]
    # ])
    points = np.asarray(points)
    # Define a reference point (e.g., the mean of all points or another specific point)
    # Here, using the mean point as the reference
    # reference_point = np.mean(points, axis=0)
    for i,reference_point in enumerate(reference_points):
        
        # Calculate the absolute error for each point
        absolute_errors = np.sum(np.abs(points - reference_point), axis=1)
        #print(absolute_errors)
        # Find the index of the point with the least absolute error
        min_error_index = np.argmin(absolute_errors)
        #print(min_error_index)
        # Get the 5D point with the least absolute error
        #print(points)
        point_with_least_error = points[min_error_index,:]
        # print(i)
        # print(point_with_least_error)
        Zpercentiles[i] = point_with_least_error[0]
        otherZpercentiles[i] = point_with_least_error[1]
        PDIpercentiles[i] = point_with_least_error[2]
        otherPDIpercentiles[i] = point_with_least_error[3]
        phipercentiles[i] = point_with_least_error[4]
    
    # print("Reference Point:", reference_point)
    # print("Point with the least absolute error:", point_with_least_error)
    # print("Minimum absolute error:", absolute_errors[min_error_index])

# reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
# nearest(data[,:5],reference_points)

# # Example usage
# data = np.array([1, 4, 5.5, 8, 10])
# targets = [5,11,12]
# nearest_value = find_nearest(data, targets)
# print(f"The nearest value to {target} is {nearest_value}.")
y = "Zl"
x = "PDI_s"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
otherZpercentiles = np.percentile(data[y], [0, 25, 50, 75, 100]) #ypercentiles
otherZpercentiles = [element for element in otherZpercentiles for _ in range(5)]
# Zpercentiles = Zpercentilesr
PDIpercentiles = np.percentile(data[x], [0, 25, 50, 75, 100]) #xpercentiles
PDIpercentiles = np.tile(PDIpercentiles, 5)
# print(PDIpercentilesr)
# PDIpercentiles = PDIpercentilesr
#other percentiles 
phipercentiles = np.percentile(data["phiL"], [0, 25, 50, 75, 100]) #xpercentiles
Zpercentiles = np.percentile(data["Zs"], [0, 25, 50, 75, 100]) #ypercentiles
otherPDIpercentiles = np.percentile(data["PDI_l"], [0, 25, 50, 75, 100]) #xpercentiles
# print(otherZpercentiles[2])
Zpercentiles = [Zpercentiles[2]] * 25
# print(otherZpercentiles)
otherPDIpercentiles = [otherPDIpercentiles[2]] * 25
phipercentiles = [phipercentiles[2]] * 25
# print(Zpercentiles)
# print(data["Z"])
# Zpercentiles = find_nearest(data[y], Zpercentiles)
# PDIpercentiles = find_nearest(data[x], PDIpercentiles)
# otherZpercentiles = find_nearest(data["Zl"], otherZpercentiles)
# otherPDIpercentiles = find_nearest(data["PDI_l"], otherPDIpercentiles)
# phipercentiles = find_nearest(data["phiL"], phipercentiles)
# print(len(Zpercentiles), len(otherZpercentiles),len( PDIpercentiles), len(otherPDIpercentiles), len(phipercentiles))
reference_points = np.column_stack((Zpercentiles, otherZpercentiles, PDIpercentiles, otherPDIpercentiles, phipercentiles))
d = data.iloc[:,:5]
nearest(d,reference_points)
# print(Zpercentiles[5], otherZpercentiles[5], PDIpercentiles[5], otherPDIpercentiles[5], phipercentiles[5])
# print(Zpercentiles)
terms = ["Very Low","Low","Mid","High","Very High"]
for term1,i in enumerate(Zpercentiles):
    # needrowGprime = pd.DataFrame()
    # needrowGdprime = pd.DataFrame()
    needrowGprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 5:100]
    needrowGdprime = data[data["Zs"]==Zpercentiles[term1]][data["PDI_s"]==PDIpercentiles[term1]][data["Zl"]==otherZpercentiles[term1]][data["PDI_l"]==otherPDIpercentiles[term1]][data["phiL"]==phipercentiles[term1]].iloc[:, 100:]
    # print(needrowGprime)
    # print(needrowGdprime)
    # print(data[data["Z"]==i].iloc[:5, :7])
    # print(data[data[" PDI"]==j].iloc[:5, :7])
    #Z errors wrt PDI
    print(len(freq),len(needrowGprime.iloc[0].values),len(needrowGdprime.iloc[0].values))
    plt.plot(freq, needrowGprime.iloc[0].values,marker='o', linestyle='-', color='red',label=f"G' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.plot(freq, needrowGdprime.iloc[0].values,marker='o', linestyle='-', color='b',label=f"G'' curve with {terms[int(term1/5)]} {y} and {terms[term1%5]} {x}")
    plt.xlabel(r'$\overline{\omega}$')
    # naming the y axis
    plt.ylabel(r"$\overline{G'} \text{ and } \overline{G''}$")
     
    # giving a title to my graph
    plt.title(f"Flow Curve at {terms[int(term1/5)]} $\\overline{{{dict[y]}}}$ and {terms[term1%5]} $\\overline{{{dict[x]}}}$")
    plt.text(min(freq), max(max(needrowGprime.iloc[0].values), max(needrowGdprime.iloc[0].values))-1, f"$\\overline{{{dict[y]}}} = {round(Zpercentiles[term1],2)}$  , $\\overline{{{dict[x]}}} = {round(PDIpercentiles[term1],2)}$  ", fontsize=10, color='black')
    # Save the plot to a file
    plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(int(term1/5),term1%5,y,x), format='png',dpi = 300)  # Save as PNG file #dpi = 600
    plt.show()
    # print(data)


In [ ]:
#Zs vs PDI_s
y = "Zs"
x = "PDI_s"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the number of rows and columns for the grid
rows, cols = 5, 5

# Create a figure with subplots arranged in a grid
fig, axs = plt.subplots(rows, cols, figsize=(12, 10))

# Load and display each image in the grid
for i in range(rows):
    for j in range(cols):
        img_path = 'C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(abs(4 - i),j,y,x)  # Path to the image file
        img = mpimg.imread(img_path)  # Load the image
        axs[i, j].imshow(img)  # Display the image
        axs[i, j].axis('off')  # Hide the axis

# Add overall x-axis and y-axis labels
fig.supxlabel(f'$\\overline{{{dict[x]}}} \longrightarrow$', fontsize=15)
fig.supylabel(f'$\\overline{{{dict[y]}}} \longrightarrow$', fontsize=15)
fig.suptitle(f'Flow curves at different values of $\\overline{{{dict[y]}}}$ and $\\overline{{{dict[x]}}}$', fontsize=15)
# # Adjust layout to prevent overlap
# plt.tight_layout()  # Adjust bottom and top margins #rect=[0, 0, 1, 0.96]
# Adjust layout to prevent overlap and increase spacing
# plt.subplots_adjust(wspace=0, hspace=0)  # Adjust space between subplots
xspace = 0.06
yspace = 0.04
space = 0.0001
plt.subplots_adjust(left=xspace, right=1-xspace, top=1-yspace, bottom=yspace, wspace=space, hspace=space)
# Adding a note using fig.text
fig.text(0.79, 0.01, f"(Note: Red curve denotes $\\overline{{G'}}$ and Blue curve denotes $\\overline{{G''}})$", ha='center', fontsize=9)
plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\allgraph1.png', format='png',dpi=600)  # Save as PNG file
plt.show()  # Display the grid of images


In [ ]:
#Zl vs PDI_s
y = "Zl"
x = "PDI_s"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the number of rows and columns for the grid
rows, cols = 5, 5

# Create a figure with subplots arranged in a grid
fig, axs = plt.subplots(rows, cols, figsize=(12, 10))

# Load and display each image in the grid
for i in range(rows):
    for j in range(cols):
        img_path = 'C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(abs(4 - i),j,y,x)  # Path to the image file
        img = mpimg.imread(img_path)  # Load the image
        axs[i, j].imshow(img)  # Display the image
        axs[i, j].axis('off')  # Hide the axis

# Add overall x-axis and y-axis labels
fig.supxlabel(f'$\\overline{{{dict[x]}}} \longrightarrow$', fontsize=15)
fig.supylabel(f'$\\overline{{{dict[y]}}} \longrightarrow$', fontsize=15)
fig.suptitle(f'Flow curves at different values of $\\overline{{{dict[y]}}}$ and $\\overline{{{dict[x]}}}$', fontsize=15)
# # Adjust layout to prevent overlap
# plt.tight_layout()  # Adjust bottom and top margins #rect=[0, 0, 1, 0.96]
# Adjust layout to prevent overlap and increase spacing
# plt.subplots_adjust(wspace=0, hspace=0)  # Adjust space between subplots
xspace = 0.06
yspace = 0.04
space = 0.0001
plt.subplots_adjust(left=xspace, right=1-xspace, top=1-yspace, bottom=yspace, wspace=space, hspace=space)
# Adding a note using fig.text
fig.text(0.79, 0.01, f"(Note: Red curve denotes $\\overline{{G'}}$ and Blue curve denotes $\\overline{{G''}})$", ha='center', fontsize=9)
plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\allgraph2.png', format='png',dpi=300)  # Save as PNG file
plt.show()  # Display the grid of images


In [ ]:
#Zs vs PDI_l
y = "Zs"
x = "PDI_l"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the number of rows and columns for the grid
rows, cols = 5, 5

# Create a figure with subplots arranged in a grid
fig, axs = plt.subplots(rows, cols, figsize=(12, 10))

# Load and display each image in the grid
for i in range(rows):
    for j in range(cols):
        img_path = 'C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(abs(4 - i),j,y,x)  # Path to the image file
        img = mpimg.imread(img_path)  # Load the image
        axs[i, j].imshow(img)  # Display the image
        axs[i, j].axis('off')  # Hide the axis

# Add overall x-axis and y-axis labels
fig.supxlabel(f'$\\overline{{{dict[x]}}} \longrightarrow$', fontsize=15)
fig.supylabel(f'$\\overline{{{dict[y]}}} \longrightarrow$', fontsize=15)
fig.suptitle(f'Flow curves at different values of $\\overline{{{dict[y]}}}$ and $\\overline{{{dict[x]}}}$', fontsize=15)
# # Adjust layout to prevent overlap
# plt.tight_layout()  # Adjust bottom and top margins #rect=[0, 0, 1, 0.96]
# Adjust layout to prevent overlap and increase spacing
# plt.subplots_adjust(wspace=0, hspace=0)  # Adjust space between subplots
xspace = 0.06
yspace = 0.04
space = 0.0001
plt.subplots_adjust(left=xspace, right=1-xspace, top=1-yspace, bottom=yspace, wspace=space, hspace=space)
# Adding a note using fig.text
fig.text(0.79, 0.01, f"(Note: Red curve denotes $\\overline{{G'}}$ and Blue curve denotes $\\overline{{G''}})$", ha='center', fontsize=9)
plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\allgraph3.png', format='png',dpi=300)  # Save as PNG file
plt.show()  # Display the grid of images


In [ ]:
#Zl vs PDI_l
y = "Zl"
x = "PDI_l"
dict = {"Zs" : "Z_s","Zl" : "Z_l","PDI_s" : "PDI_s", "PDI_l" : "PDI_l", "phiL" : "\phi_l"}
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the number of rows and columns for the grid
rows, cols = 5, 5

# Create a figure with subplots arranged in a grid
fig, axs = plt.subplots(rows, cols, figsize=(12, 10))

# Load and display each image in the grid
for i in range(rows):
    for j in range(cols):
        img_path = 'C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\graph{2}{3}_{0}{1}.png'.format(abs(4 - i),j,y,x)  # Path to the image file
        img = mpimg.imread(img_path)  # Load the image
        axs[i, j].imshow(img)  # Display the image
        axs[i, j].axis('off')  # Hide the axis

# Add overall x-axis and y-axis labels
fig.supxlabel(f'$\\overline{{{dict[x]}}} \longrightarrow$', fontsize=15)
fig.supylabel(f'$\\overline{{{dict[y]}}} \longrightarrow$', fontsize=15)
fig.suptitle(f'Flow curves at different values of $\\overline{{{dict[y]}}}$ and $\\overline{{{dict[x]}}}$', fontsize=15)
# # Adjust layout to prevent overlap
# plt.tight_layout()  # Adjust bottom and top margins #rect=[0, 0, 1, 0.96]
# Adjust layout to prevent overlap and increase spacing
# plt.subplots_adjust(wspace=0, hspace=0)  # Adjust space between subplots
xspace = 0.06
yspace = 0.04
space = 0.0001
plt.subplots_adjust(left=xspace, right=1-xspace, top=1-yspace, bottom=yspace, wspace=space, hspace=space)
# Adding a note using fig.text
fig.text(0.79, 0.01, f"(Note: Red curve denotes $\\overline{{G'}}$ and Blue curve denotes $\\overline{{G''}})$", ha='center', fontsize=9)
plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\bigraphs\\allgraph4.png', format='png',dpi=600)  # Save as PNG file
plt.show()  # Display the grid of images


In [ ]:
data.describe(include = "all")

In [ ]:
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    plt.hist(y[s])
    plt.show()
    # print(y[s].min())
    # print(y[s].max())

In [ ]:
d = data.iloc[:,100:]
print(d.head(5))
d = d.values.flatten()
d
print(d.min())
print(d.max())

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(128, input_shape=(1,190),return_sequences=True)) #return_sequences=True
    #model.add(Dropout(0.1))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(32))
    #model.add(Dropout(0.05))
    #model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    #model.add(Dropout(0.05))
    #model.add(LSTM(16))
    model.add(Dense(5, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.006788847502321005

In [ ]:
#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df 

In [ ]:
df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
Zs_mae

#Best Zs_mae (RNN) = 0.036031487999376444

In [ ]:
df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
Zl_mae

#Best Zl_mae (RNN) = 0.026321348207570663

In [ ]:
df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
PDI_s_mae

#Best PDI_s_mae (RNN) = 0.027779294553068122


In [ ]:
df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
PDI_l_mae

#Best PDI_s_mae (RNN) = 0.02168924099052297

In [ ]:
df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
phiL_mae

#Best phiL_mae (RNN) = 0.0390145898566803

In [ ]:
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal.csv', index=False)

In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal.h5")

In [ ]:
#Code 15 - Error Analysis on Bimodal model

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\bi_PE_5param_wint_2_400000_training.csv")  # Replace with your CSV file path

In [ ]:
X = data.iloc[:, 5:]  # 190 explanatory variables
y = data.iloc[:, :5]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
# Load the model
loaded_model = tf.keras.models.load_model("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal.h5")

X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Make predictions with the loaded model
y_pred_loaded = loaded_model.predict(X_test_reshaped)

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred_loaded,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

In [ ]:
df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.06409748865866961

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.04218965152593366

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.04768009408433895

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.036523118682293854

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.06807153102156327

In [ ]:
print({Zs_mae*100/(max(df["Zs"]) - min(df["Zs"]))})
print({PDI_s_mae*100/(max(df["PDI_s"]) - min(df["PDI_s"]))})
print({Zl_mae*100/(max(df["Zl"]) - min(df["Zl"]))})
print({PDI_l_mae*100/(max(df["PDI_l"]) - min(df["PDI_l"]))})
print({phiL_mae*100/(max(df["phiL"]) - min(df["phiL"]))})

In [ ]:
print(max(df["PDI_lPRED"]))


In [ ]:
df.describe(include="all")

In [ ]:
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(f'{s} : {min(df[s+"errors"]))} , {max(df[s+"errors"])}')

In [ ]:
for s in strings:
    print(f'{s} : {len(df[df[s+"errors"]>0.06][s+"errors"])*100/len(df[s+"errors"])}')

In [ ]:
# plotting the points Zs
plt.scatter(df["Zs"], df["Zserrors"])
 
# naming the x axis
plt.xlabel('Zs')
# naming the y axis
plt.ylabel('Zs Errors')
 
# giving a title to my graph
plt.title('Zs Errors')
 
# function to show the plot
plt.show()

# plotting the points Zl
plt.scatter(df["Zl"], df["Zlerrors"])
 
# naming the x axis
plt.xlabel('Zl')
# naming the y axis
plt.ylabel('Zl Errors')
 
# giving a title to my graph
plt.title('Zl Errors')
 
# function to show the plot
plt.show()

# plotting the points PDI_s
plt.scatter(df["PDI_s"], df["PDI_serrors"])
 
# naming the x axis
plt.xlabel('PDI_s')
# naming the y axis
plt.ylabel('PDI_s Errors')
 
# giving a title to my graph
plt.title('PDI_s Errors')
 
# function to show the plot
plt.show()

# plotting the points PDI_l
plt.scatter(df["PDI_l"], df["PDI_lerrors"])
 
# naming the x axis
plt.xlabel('PDI_l')
# naming the y axis
plt.ylabel('PDI_l Errors')
 
# giving a title to my graph
plt.title('PDI_l Errors')
 
# function to show the plot
plt.show()

# plotting the points 
plt.scatter(df["phiL"], df["phiLerrors"])
 
# naming the x axis
plt.xlabel('phiL')
# naming the y axis
plt.ylabel('phiL Errors')
 
# giving a title to my graph
plt.title('phiL Errors')
 
# function to show the plot
plt.show()


In [ ]:
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(str(min(df[s])) + " , " + str(max(df[s])))

In [ ]:
POINTS = [[]]
NEWDATA = [[]]
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]
names = ["Z_s","Z_l","PDI_s","PDI_l","\phi_l"]

for k,s in enumerate(strings):

    for j, S in enumerate(strings):
        val = min(df[s])
        points = [val]
        while val + 0.125 < max(df[s]):
            val = val + 0.125
            points.append(val)
        points.append(max(df[s]))
    
        #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
        newdata = []
        for i,c in enumerate(points[:-1]):
            if i!=len(points)-1:
                arr = df[df[s]>=points[i]][df[s]<points[i+1]][S+"errors"]
                newdata.append(sum(arr)/len(arr))
            else:
                arr = df[df[s]>=points[i]][df[s]<=points[i+1]][S+"errors"]
                newdata.append(sum(arr)/len(arr))
        newdata
        
        points2 = [x + 0.0625 for x in points[:-1]]
        plt.plot(points2, newdata,marker='o', linestyle='-', color='purple')
        # naming the x axis
        plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
        # naming the y axis
        plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
         
        # giving a title to my graph
        plt.title(r'$\overline{{{1}}} \text{{ Errors vs }} \overline{{{0}}}$'.format(names[k], names[j]))
        plt.show()
        #plt.plot(points2, newdata)

        #POINTS.append(points2)
        #NEWDATA.append(newdata)
        
        plt.figure(figsize=(7.5, 6))
        #plt.figure(figsize=(5, 4))
        
        # # Plot the density heatmap
        # sns.kdeplot(x=df[s], y=df[s+"errors"], cmap='Spectral', fill=False, thresh=0, levels=10)
        
        # # plotting the points 
        # plt.scatter(df[s], df[s+"errors"],s=0.1, c='black', alpha=0.5)
        
        # # Plot the density heatmap
        # #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
        # plt.plot(points2, newdata)
    
        median_x = np.median(df[s])
        median_y = np.median(df[S+"errors"])
        
        Q1_x, Q3_x = np.percentile(df[s], [25, 75])
        Q1_y, Q3_y = np.percentile(df[S+"errors"], [25, 75])
        
        IQR_x = Q3_x - Q1_x
        IQR_y = Q3_y - Q1_y
        # Plot the density heatmap
        sns.kdeplot(x=df[s], y=df[S+"errors"], cmap='Spectral', fill=False, thresh=0.05, levels=2)
        
        # plotting the points 
        plt.scatter(df[s], df[S+"errors"],s=0.1, c='black', alpha=0.5)
        
        plt.axhline(median_y, color='red', linestyle='--', linewidth=0.5, label='Median Y')
        # plt.axvline(median_x, color='red', linestyle='--', linewidth=0.5, label='Median X')
        plt.axhline(Q1_y, color='red', linestyle='--', linewidth=0.5, label='1st Quartile Y')
        plt.axhline(Q3_y, color='red', linestyle='--', linewidth=0.5, label='3rd Quartile Y')
        # Plot the density heatmap
        #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
        plt.plot(points2, newdata,marker='o', linestyle='-', color='lime')
        plt.xlim(df[s].min()-0.3, df[s].max()+0.3)  # Set the y-axis limit from 0 to 1
        #plt.ylim(0, 0.06)  # Set the y-axis limit from 0 to 1
        
        # naming the x axis
        plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
        # naming the y axis
        plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
         
        # giving a title to my graph
        plt.title(r'$\overline{{{0}}} \text{{ Errors with Density Heatmap }}$'.format(names[j]))
         
        # function to show the plot
        plt.show()

In [ ]:
def code2():
    POINTS = [[]]
    NEWDATA = [[]]
    strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]
    names = ["Z_s","Z_l","PDI_s","PDI_l","\phi_l"]
    
    for k,s in enumerate(strings):
    
        for j, S in enumerate(strings):
            val = min(df[s])
            points = [val]
            while val + 0.125 < max(df[s]):
                val = val + 0.125
                points.append(val)
            points.append(max(df[s]))
        
            #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
            newdata = []
            for i,c in enumerate(points[:-1]):
                if i!=len(points)-1:
                    arr = df[df[s]>=points[i]][df[s]<points[i+1]][S+"errors"]
                    newdata.append(sum(arr)/len(arr))
                else:
                    arr = df[df[s]>=points[i]][df[s]<=points[i+1]][S+"errors"]
                    newdata.append(sum(arr)/len(arr))
            newdata
            
            points2 = [x + 0.0625 for x in points[:-1]]
            plt.plot(points2, newdata,marker='o', linestyle='-', color='purple')
            # naming the x axis
            plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
            # naming the y axis
            plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
             
            # giving a title to my graph
            plt.title(r'$\overline{{{1}}} \text{{ Errors vs }} \overline{{{0}}}$'.format(names[k], names[j]))
            plt.show()
            #plt.plot(points2, newdata)
    
            #POINTS.append(points2)
            #NEWDATA.append(newdata)
            
            plt.figure(figsize=(7.5, 6))
            #plt.figure(figsize=(5, 4))
            
            # # Plot the density heatmap
            # sns.kdeplot(x=df[s], y=df[s+"errors"], cmap='Spectral', fill=False, thresh=0, levels=10)
            
            # # plotting the points 
            # plt.scatter(df[s], df[s+"errors"],s=0.1, c='black', alpha=0.5)
            
            # # Plot the density heatmap
            # #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
            # plt.plot(points2, newdata)
        
            median_x = np.median(df[s])
            median_y = np.median(df[S+"errors"])
            
            Q1_x, Q3_x = np.percentile(df[s], [25, 75])
            Q1_y, Q3_y = np.percentile(df[S+"errors"], [25, 75])
            
            IQR_x = Q3_x - Q1_x
            IQR_y = Q3_y - Q1_y
            # Plot the density heatmap
            sns.kdeplot(x=df[s], y=df[S+"errors"], cmap='Spectral', fill=False, thresh=0.05, levels=2)
            
            # plotting the points 
            plt.scatter(df[s], df[S+"errors"],s=0.1, c='black', alpha=0.5)
            
            plt.axhline(median_y, color='red', linestyle='--', linewidth=0.5, label='Median Y')
            # plt.axvline(median_x, color='red', linestyle='--', linewidth=0.5, label='Median X')
            plt.axhline(Q1_y, color='red', linestyle='--', linewidth=0.5, label='1st Quartile Y')
            plt.axhline(Q3_y, color='red', linestyle='--', linewidth=0.5, label='3rd Quartile Y')
            # Plot the density heatmap
            #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
            plt.plot(points2, newdata,marker='o', linestyle='-', color='lime')
            plt.xlim(df[s].min()-0.3, df[s].max()+0.3)  # Set the y-axis limit from 0 to 1
            #plt.ylim(0, 0.06)  # Set the y-axis limit from 0 to 1
            
            # naming the x axis
            plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
            # naming the y axis
            plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
             
            # giving a title to my graph
            plt.title(r'$\overline{{{0}}} \text{{ Errors with Density Heatmap }}$'.format(names[j]))
             
            # function to show the plot
            plt.show()

In [ ]:
def code3(bestname):
    POINTS = [[]]
    NEWDATA = [[]]
    strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]
    names = ["Z_s","Z_l","PDI_s","PDI_l","\phi_l"]
    
    for k,s in enumerate(strings):
    
        for j, S in enumerate(strings):
            val = min(df[s])
            points = [val]
            while val + 0.125 < max(df[s]):
                val = val + 0.125
                points.append(val)
            points.append(max(df[s]))
        
            #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
            newdata = []
            for i,c in enumerate(points[:-1]):
                if i!=len(points)-1:
                    arr = df[df[s]>=points[i]][df[s]<points[i+1]][S+"errors"]
                    newdata.append(sum(arr)/len(arr))
                else:
                    arr = df[df[s]>=points[i]][df[s]<=points[i+1]][S+"errors"]
                    newdata.append(sum(arr)/len(arr))
            newdata
            
            points2 = [x + 0.0625 for x in points[:-1]]
            plt.plot(points2, newdata,marker='o', linestyle='-', color='purple')
            # naming the x axis
            plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
            # naming the y axis
            plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
             
            # giving a title to my graph
            plt.title(r'$\overline{{{1}}} \text{{ Errors vs }} \overline{{{0}}}$'.format(names[k], names[j]))
            plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\binormal\\{2}_graph_{0}{1}.png'.format(k,j,bestname), format='png',dpi=600)  # Save as PNG file
            plt.show()
            
            # plt.figure(figsize=(7.5, 6))
        
            # median_x = np.median(df[s])
            # median_y = np.median(df[S+"errors"])
            
            # Q1_x, Q3_x = np.percentile(df[s], [25, 75])
            # Q1_y, Q3_y = np.percentile(df[S+"errors"], [25, 75])
            
            # IQR_x = Q3_x - Q1_x
            # IQR_y = Q3_y - Q1_y
            # # Plot the density heatmap
            # sns.kdeplot(x=df[s], y=df[S+"errors"], cmap='Spectral', fill=False, thresh=0.05, levels=2)
            
            # # plotting the points 
            # plt.scatter(df[s], df[S+"errors"],s=0.1, c='black', alpha=0.5)
            
            # plt.axhline(median_y, color='red', linestyle='--', linewidth=0.5, label='Median Y')
            # # plt.axvline(median_x, color='red', linestyle='--', linewidth=0.5, label='Median X')
            # plt.axhline(Q1_y, color='red', linestyle='--', linewidth=0.5, label='1st Quartile Y')
            # plt.axhline(Q3_y, color='red', linestyle='--', linewidth=0.5, label='3rd Quartile Y')
            # # Plot the density heatmap
            # #sns.kdeplot(x=df["Z"], y=df["zerrors"], cmap="Blues", fill=True, thresh=0, levels=100)
            # plt.plot(points2, newdata,marker='o', linestyle='-', color='lime')
            # plt.xlim(df[s].min()-0.3, df[s].max()+0.3)  # Set the y-axis limit from 0 to 1
            # #plt.ylim(0, 0.06)  # Set the y-axis limit from 0 to 1
            
            # # naming the x axis
            # plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
            # # naming the y axis
            # plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
             
            # # giving a title to my graph
            # plt.title(r'$\overline{{{0}}} \text{{ Errors with Density Heatmap }}$'.format(names[j]))
             
            # # function to show the plot
            # plt.show()

In [ ]:
code3("original")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
def code4(bestname):
    
    # Define the number of rows and columns for the grid
    rows, cols = 5, 5
    
    # Create a figure with subplots arranged in a grid
    fig, axs = plt.subplots(rows, cols, figsize=(12, 10))
    
    # Define the labels for rows and columns
    labels = [r"$\overline{\phi_l}$", r"$\overline{PDI_l}$", r"$\overline{PDI_s}$", r"$\overline{Z_l}$",r"$\overline{Z_s}$"]
    
    # Load and display each image in the grid
    for i in range(rows):
        for j in range(cols):
            img_path = 'C:\\Users\\Kevin\\Desktop\\Dissertation\\binormal\\{2}_graph_{0}{1}.png'.format(j, abs(4 - i), bestname)  # Path to the image file
            img = mpimg.imread(img_path)  # Load the image
            axs[i, j].imshow(img)  # Display the image
            axs[i, j].axis('off')  # Hide the axis
    
    # Add row and column labels
    for i, label in enumerate(labels):
        # Add labels to the left of each row
        fig.text(0.04, 0.87 - i * 0.1785, label, va='center', ha='center', fontsize=12, rotation='horizontal')
        # Add labels above each column
        fig.text(0.15 + i * 0.178, 0.037, labels[len(labels)-1-i], va='center', ha='center', fontsize=12) #0.92
    # # Add overall x-axis and y-axis labels
    # fig.supxlabel(r'$\overline{PDI} \longrightarrow$', fontsize=15)
    # fig.supylabel(r'$\overline{Z} \longrightarrow$', fontsize=15)
    # fig.suptitle(f'Flow curves at different values of $\\overline{{Z}}$ and $\\overline{{PDI}}$', fontsize=15)
    
    # Adjust layout to prevent overlap and increase spacing
    xspace = 0.06
    yspace = 0.04
    space = 0.0001
    plt.subplots_adjust(left=xspace, right=1-xspace, top=1-yspace, bottom=yspace, wspace=space, hspace=space)
    
    # Adding a note using fig.text
    # fig.text(0.79, 0.01, f"(Note: Red curve denotes $\\overline{{G'}}$ and Blue curve denotes $\\overline{{G''}})$", ha='center', fontsize=9)
    
    # Save and show the plot
    plt.savefig('C:\\Users\\Kevin\\Desktop\\Dissertation\\binormal\\allgraph.png', format='png', dpi=600)  # Save as PNG file
    plt.show()  # Display the grid of images


In [ ]:
code4("original")

In [ ]:
def code5(bestthing):
    code3(bestthing)
    code4(bestthing)

In [ ]:
#Code 16 - Modelling bimodal with restricted freq range

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\bi_PE_5param_wint_2_400000_training.csv")  # Replace with your CSV file path

#X = data.iloc[:, 5:]  # 190 explanatory variables

X1 = data.iloc[:, 22+3:49+3]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = data.iloc[:, 117+3:144+3]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)

y = data.iloc[:, :5]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(128, input_shape=(1,54),return_sequences=True)) #return_sequences=True
    #model.add(Dropout(0.1))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(32))
    #model.add(Dropout(0.05))
    #model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    #model.add(Dropout(0.05))
    #model.add(LSTM(16))
    model.add(Dense(5, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=128, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.006788847502321005

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.036031487999376444

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.026321348207570663

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.027779294553068122

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.02168924099052297

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.0390145898566803



In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal_restricted.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_restricted.csv', index=False)

In [ ]:
#Code 17 - Error analysis of Restricted Bimodal thing

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

df = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_restricted.csv")  # Replace with your CSV file path

In [ ]:
code2()

In [ ]:
code5("restricted")

In [ ]:
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(str(min(df[s])) + " , " + str(max(df[s])))

df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.06409748865866961

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.04218965152593366

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.04768009408433895

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.036523118682293854

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.06807153102156327

print({Zs_mae*100/(max(df["Zs"]) - min(df["Zs"]))})
print({PDI_s_mae*100/(max(df["PDI_s"]) - min(df["PDI_s"]))})
print({Zl_mae*100/(max(df["Zl"]) - min(df["Zl"]))})
print({PDI_l_mae*100/(max(df["PDI_l"]) - min(df["PDI_l"]))})
print({phiL_mae*100/(max(df["phiL"]) - min(df["phiL"]))})

strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(f'{s} : {min(df[s+"errors"])} , {max(df[s+"errors"])}')

for s in strings:
    print(f'{s} : {len(df[df[s+"errors"]>0.06][s+"errors"])*100/len(df[s+"errors"])}')

In [ ]:
#Code 18 - Modelling bimodal with inducing artificial errors

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\bi_PE_5param_wint_2_400000_training.csv")  # Replace with your CSV file path

X = data.iloc[:, 5:]  # 190 explanatory variables

noise = np.random.normal(0, 0.1, (408050, 1))
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

y = data.iloc[:, :5]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(128, input_shape=(1,190),return_sequences=True)) #return_sequences=True
    #model.add(Dropout(0.1))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(32))
    #model.add(Dropout(0.05))
    #model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    #model.add(Dropout(0.05))
    #model.add(LSTM(16))
    model.add(Dense(5, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=128, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.006788847502321005

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.036031487999376444

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.026321348207570663

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.027779294553068122

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.02168924099052297

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.0390145898566803



In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal_randomerrors.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv', index=False)

In [ ]:
#Code 19 - Error analysis of Random error prediction bimodal

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

df = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv")  # Replace with your CSV file path

In [ ]:
code2()

In [ ]:
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(str(min(df[s])) + " , " + str(max(df[s])))

df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.06409748865866961

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.04218965152593366

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.04768009408433895

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.036523118682293854

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.06807153102156327

print({Zs_mae*100/(max(df["Zs"]) - min(df["Zs"]))})
print({PDI_s_mae*100/(max(df["PDI_s"]) - min(df["PDI_s"]))})
print({Zl_mae*100/(max(df["Zl"]) - min(df["Zl"]))})
print({PDI_l_mae*100/(max(df["PDI_l"]) - min(df["PDI_l"]))})
print({phiL_mae*100/(max(df["phiL"]) - min(df["phiL"]))})

strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(f'{s} : {min(df[s+"errors"])} , {max(df[s+"errors"])}')

for s in strings:
    print(f'{s} : {len(df[df[s+"errors"]>0.06][s+"errors"])*100/len(df[s+"errors"])}')

In [ ]:
code5("artificial")

In [ ]:
#Code 20 - Modelling bimodal with pseudo real life data

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the dataset
data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\bi_PE_5param_wint_2_400000_training.csv")  # Replace with your CSV file path

X1 = data.iloc[:, 22+3:49+3]  # (G'(20) to G'(46)) (10^-1 to 10^2) (27 columns)
X2 = data.iloc[:, 117+3:144+3]  # (G''(20) to G''(46)) (10^-1 to 10^2) (27 columns)
X = pd.concat([X1, X2], axis=1)

noise = np.random.normal(0, 0.1, (408050, 1))
X = X.add(noise, axis=0) #multiplicative errors (row - wise)
noise = np.random.normal(0, 0.05, X.shape)
X = X + noise #additive errors (element wise)

y = data.iloc[:, :5]  # 2 numerical variables to predict

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xcopytest = X_test

# Standardize the data (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X_trainNP = X_train.flatten()
y_trainNP = y_train.to_numpy()

In [ ]:
#metrics=['mae']
#Define the MLP model
def create_mlp_model(input_dim):
    model = Sequential()
    #model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(LSTM(128, input_shape=(1,54),return_sequences=True)) #return_sequences=True
    #model.add(Dropout(0.1))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(32))
    #model.add(Dropout(0.05))
    #model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    #model.add(Dropout(0.05))
    #model.add(LSTM(16))
    model.add(Dense(5, activation='linear'))  # Output layer for 2 numerical variables
    return model

# Create the model
model = create_mlp_model((1,X_train.shape[1]))

#Create the Adam optimizer with a custom learning rate
learning_rate = 0.0001  # Set your desired learning rate here #10^-4 before
custom_optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['accuracy'])#metrics=['mae']

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

type(X_train)
type(y_train)

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Train the model
history = model.fit(X_train_reshaped, y_trainNP, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=128, 
                    callbacks=[early_stopping])


#Test Loss: 9.01788444025442e-05

In [ ]:
# Evaluate the model
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

loss = model.evaluate(X_test_reshaped, y_test.to_numpy())
print(f'Test Loss: {loss}')

# Make predictions
y_pred = model.predict(X_test_reshaped)

#Best Test Loss: 0.006788847502321005

#df1 = pd.DataFrame(xcopytest,columns = data.columns[2:])
df1 = xcopytest.reset_index(drop=False)
df1["index"] = df1["index"] + 2
df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
df3 = pd.DataFrame(y_pred,columns = [x+"PRED" for x in y_test.columns])
df = df1.join(df2).join(df3)   

df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.036031487999376444

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.026321348207570663

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.027779294553068122

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.02168924099052297

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.0390145898566803



In [ ]:
# # Save the model
# model.save("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal_realdata.h5")
# df.to_csv('C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_realdata.csv', index=False)

In [ ]:
#Code 21 - Error analysis of bimodal pseudo real life data predictions

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

df = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_realdata.csv")  # Replace with your CSV file path

In [ ]:
code2()

In [ ]:
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(str(min(df[s])) + " , " + str(max(df[s])))

df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
print(Zs_mae)

#Best Zs_mae (RNN) = 0.06409748865866961

df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
print(Zl_mae)

#Best Zl_mae (RNN) = 0.04218965152593366

df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
print(PDI_s_mae)

#Best PDI_s_mae (RNN) = 0.04768009408433895

df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
print(PDI_l_mae)

#Best PDI_s_mae (RNN) = 0.036523118682293854

df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
print(phiL_mae)

#Best phiL_mae (RNN) = 0.06807153102156327

print({Zs_mae*100/(max(df["Zs"]) - min(df["Zs"]))})
print({PDI_s_mae*100/(max(df["PDI_s"]) - min(df["PDI_s"]))})
print({Zl_mae*100/(max(df["Zl"]) - min(df["Zl"]))})
print({PDI_l_mae*100/(max(df["PDI_l"]) - min(df["PDI_l"]))})
print({phiL_mae*100/(max(df["phiL"]) - min(df["phiL"]))})

strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]

for s in strings:
    print(f'{s} : {min(df[s+"errors"])} , {max(df[s+"errors"])}')

for s in strings:
    print(f'{s} : {len(df[df[s+"errors"]>0.06][s+"errors"])*100/len(df[s+"errors"])}')

In [ ]:
code5("pseudo")

In [ ]:
#All the graphs together - 

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
loadings = ["C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_restricted.csv",
            "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv","C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_realdata.csv"]

POINTS = [[]]
NEWDATA = [[]]
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]
for l in loadings:
    df = pd.read_csv(l)  # Replace with your CSV file path
    for i,s in enumerate(strings):
    
        for j, S in enumerate(strings):
            val = min(df[s])
            points = [val]
            while val + 0.125 < max(df[s]):
                val = val + 0.125
                points.append(val)
            points.append(max(df[s]))
        
            #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
            newdata = []
            for i,c in enumerate(points[:-1]):
                if i!=len(points)-1:
                    arr = df[df[s]>=points[i]][df[s]<points[i+1]][S+"errors"]
                    newdata.append(sum(arr)/len(arr))
                else:
                    arr = df[df[s]>=points[i]][df[s]<=points[i+1]][S+"errors"]
                    newdata.append(sum(arr)/len(arr))
            newdata
            
            points2 = [x + 0.0625 for x in points[:-1]]
            # plt.plot(points2, newdata,marker='o', linestyle='-', color='purple')
            # plt.title(S +' Errors'+' vs ' +s)
            # plt.xlabel(s)
            # plt.ylabel(S +' Errors')
            # plt.show()
            # #plt.plot(points2, newdata)
    
            POINTS.append(points2)
            NEWDATA.append(newdata)
        

In [ ]:
#Every combination
count = 0
strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]
names = ["Z_s","Z_l","PDI_s","PDI_l","\phi_l"]
for i,s in enumerate(strings): 
    for j, S in enumerate(strings):
        plt.plot(POINTS[76+count], NEWDATA[76+count],marker='o', linestyle='-', color='purple',label='Prediction with restricted frequency and artificial errors induced (Pseudo real life data)')
        plt.plot(POINTS[51+count], NEWDATA[51+count],marker='o', linestyle='-', color='red',label='Prediction with artificial errors induced')
        plt.plot(POINTS[26+count], NEWDATA[26+count],marker='o', linestyle='-', color='b',label='Prediction with restricted frequency range')
        plt.plot(POINTS[1+count], NEWDATA[1+count],marker='o', linestyle='-', color='lime',label='Prediction on original dataset')
        plt.xlabel(r'$\overline{{{0}}}$'.format(names[i]))
        # naming the y axis
        plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[j]))
        # giving a title to my graph
        plt.title(r'$\overline{{{1}}} \text{{ Errors vs }} \overline{{{0}}}$'.format(names[i], names[j]))
        plt.legend()
        plt.show()
        count = count + 1

In [ ]:
#Code 22 - Training on dirty data and predicting on clean and vice versa

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM

import numpy as np
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

def bitrying(moodel, deteset, texts):
    # Load the dataset
    data = pd.read_csv("C:\\Users\\Kevin\\Desktop\\Dissertation\\bi_PE_5param_wint_2_400000_training.csv")  # Replace with your CSV file path
    #data[" PDI"] = np.log10((((10 ** data[" PDI"]) + 1)*1))
    
    X = data.iloc[:, 5:]  # 190 explanatory variables
    y = data.iloc[:, :5]  # 2 numerical variables to predict

    if deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv":
        noise = np.random.normal(0, 0.1, (408050, 1))
        # noise2 = np.random.normal(1, 0, (202500, 1))
        # noise3 = noise + noise2
        X = X.add(noise, axis=0) #multiplicative errors (row - wise)
        noise = np.random.normal(0, 0.05, X.shape)
        X = X + noise #additive errors (element wise)
        
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    xcopytest = X_test
    
    # Standardize the data (mean=0, variance=1)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    #X_trainNP = X_train.flatten()
    y_trainNP = y_train.to_numpy()
    
    # Load the model
    loaded_model = tf.keras.models.load_model(moodel)
    
    X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    # Make predictions with the loaded model
    y_pred_loaded = loaded_model.predict(X_test_reshaped)
    
    df1 = pd.DataFrame(xcopytest,columns = X.columns)
    df1 = df1.reset_index(drop=False)
    df1["index"] = df1["index"] + 2
    df2 = pd.DataFrame(y_test.to_numpy(),columns = [x for x in y_test.columns])
    df3 = pd.DataFrame(y_pred_loaded,columns = [x+"PRED" for x in y_test.columns])
    df = df1.join(df2).join(df3)

    df["Zserrors"] = (df["Zs"] - df["ZsPRED"]).abs()
    Zs_mae =sum(df["Zserrors"])/len(df["Zserrors"])
    print(Zs_mae)
    
    #Best Zs_mae (RNN) = 0.036031487999376444
    
    df["Zlerrors"] = (df["Zl"] - df["ZlPRED"]).abs()
    Zl_mae =sum(df["Zlerrors"])/len(df["Zlerrors"])
    print(Zl_mae)
    
    #Best Zl_mae (RNN) = 0.026321348207570663
    
    df["PDI_serrors"] = (df["PDI_s"] - df["PDI_sPRED"]).abs()
    PDI_s_mae =sum(df["PDI_serrors"])/len(df["PDI_serrors"])
    print(PDI_s_mae)
    
    #Best PDI_s_mae (RNN) = 0.027779294553068122
    
    df["PDI_lerrors"] = (df["PDI_l"] - df["PDI_lPRED"]).abs()
    PDI_l_mae =sum(df["PDI_lerrors"])/len(df["PDI_lerrors"])
    print(PDI_l_mae)
    
    #Best PDI_s_mae (RNN) = 0.02168924099052297
    
    df["phiLerrors"] = (df["phiL"] - df["phiLPRED"]).abs()
    phiL_mae =sum(df["phiLerrors"])/len(df["phiLerrors"])
    print(phiL_mae)

    #Best phiL_mae (RNN) = 0.0390145898566803
    
    if deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv":
        deteset2 = "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal.csv"
    elif deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal.csv":
        deteset2 = "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv"

    loadings = [0,deteset2]
    count = 0
    NEWDATA = [[]]
    POINTS = [[]]
    strings = ["Zs","Zl","PDI_s","PDI_l","phiL"]
    names = ["Z_s","Z_l","PDI_s","PDI_l","\phi_l"]
    for l in loadings:
        if l!=0:
            df = pd.read_csv(l)
        for i,s in enumerate(strings):
        
                for j,S in enumerate(strings):
                    val = min(df[s])
                    points = [val]
                    while val + 0.0625*2 < max(df[s]):
                        val = val + 0.0625*2
                        points.append(val)
                    points.append(max(df[s]))
                
                    #points = [min(df[" PDI"]),-1,-0.5,0,0.5,max(df[" PDI"])]
                    newdata = []
                    for i,c in enumerate(points[:-1]):
                        if i!=len(points)-1:
                            arr = df[df[s]>=points[i]][df[s]<points[i+1]][S+"errors"]
                            newdata.append(sum(arr)/len(arr))
                        else:
                            arr = df[df[s]>=points[i]][df[s]<=points[i+1]][S+"errors"]
                            newdata.append(sum(arr)/len(arr))
                    newdata
                    #newdataZ = newdata
                    points = [x + 0.0625 for x in points[:-1]]
                    POINTS.append(points)
                    NEWDATA.append(newdata)
                    #plt.plot(points, newdata,marker='o', linestyle='-', color='lime')
                    #plt.show()
        
    if deteset == "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv":
        extra = 0
    else:
        extra = 2

    for k,ss in enumerate(strings):
        for kk,sss in enumerate(strings):
            plt.plot(POINTS[1+k*kk], NEWDATA[1+k*kk],marker='o', linestyle='-', color='red',label=texts[0+extra])
            plt.plot(POINTS[26+k*kk], NEWDATA[26+k*kk],marker='o', linestyle='-', color='lime',label=texts[1+extra])
            plt.title(sss+' Errors vs '+ss)
            plt.xlabel(ss)
            plt.ylabel(sss+' Errors')
            plt.xlabel(r'$\overline{{{0}}}$'.format(names[k]))
            # naming the y axis
            plt.ylabel(r'$\overline{{{0}}} \text{{ Errors}}$'.format(names[kk]))
            # giving a title to my graph
            plt.title(r'$\overline{{{1}}} \text{{ Errors vs }} \overline{{{0}}}$'.format(names[k], names[kk]))
            plt.legend()
            plt.show()
        

In [ ]:
textss = ["Prediction of clean model on unclean data","Prediction of clean model on clean data","Prediction of unclean model on clean data","Prediction of unclean model on unclean data"]

In [ ]:
bitrying("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal.h5", "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal_randomerrors.csv",textss)

In [ ]:
bitrying("C:\\Users\\Kevin\\Desktop\\Dissertation\\mlp_model_binormal_randomerrors.h5", "C:\\Users\\Kevin\\Desktop\\Dissertation\\out_binormal.csv",textss)

In [ ]:
#Hopefully the end :_)